
TODO:
better lemma. Ok did a tiny bit of this. Could support unification variables now too.

Protocol based generic reasoning
Do software foundations.
Semantical Refinement typing
Hoare

Proof certs

Look at all the stuff I got stumped on. Is it easy somehow it Lean?
Using lean as smt


https://github.com/ftsrg/chc2c
extraction

https://arxiv.org/abs/2505.13454 pyeb: A Python Implementation of Event-B Refinement Calculus https://github.com/ncatanoc/pyeb

Compete in smtlib / cade?

TPTP parser? 

Try ingesting all of TPTP problem set. Egraph?


float16 in numpy. Brute force check properties of floats.
float8 would be a lookup table maybe.



Should I be doing more with triggers?

# curry howard


# monad modules

algerbaic effects?


https://www.isa-afp.org/browser_info/current/AFP/Monomorphic_Monad/outline.pdf  Effect Polymorphism in Higher-Order Logic
Andreas Lochbihler http://www.andreas-lochbihler.de/pub/lochbihler2017itp.pdf




In [ ]:
class OfInt(Protocol):
    T : smt.SortRef
    from_int : smt.FuncDeclRef
class OfReal(Protocol):
    T : smt.SortRef
    from_real : smt.FuncDeclRef
class OfBool(Protocol):
    T : smt.SortRef
    from_bool : smt.FuncDeclRef
class Container(Protocol): # better name?
    T : smt.SortRef
    Elt : smt.SortRef
class Monad(Protocol, Applicative): # monomorphic monad
    T : smt.SortRef
    Elt : smt.SortRef
    bind : smt.FuncDeclRef # (T, Elt -> T) -> T
class Functor(Protocol):
    T : smt.SortRef
    Elt : smt.SortRef
    fmap : smt.FuncDeclRef # (Elt -> Elt) -> T -> T
class Applicative(Protocol, Functor):
    T : smt.SortRef
    Elt : smt.SortRef
    pure : smt.FuncDeclRef # Elt -> T
    ap : smt.FuncDeclRef # (T, T) -> T'

class LawfulMonad(Protocol, Monad):
    T : smt.SortRef
    Elt : smt.SortRef
    bind_assoc : kd.Proof  # bind (bind m f) g == bind m (x -> bind (f x) g)
    return_bind : kd.Proof # bind (pure x) f == f x
    bind_return : kd.Proof # bind m pure == m

# Z3 Arena
I think that rebuilding the wrapper costs a lot of time.
If i had a managed thing that didn't do that



In [3]:
import z3
x = z3.Int("x")

In [4]:
%%timeit
x.eq(x)

716 ns ± 5.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
%%timeit
x == x

20.8 μs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
x.ctx.ctx

In [19]:
%%timeit
z3.Z3_mk_eq(x.ctx.ctx, x.as_ast(), x.as_ast())

768 ns ± 3.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
class Arena():
    def __init__(self, ctx=None):
        if ctx is not None:
            self.ctx = ctx.ctx
        else:
            self.ctx = z3.main_ctx().ctx
        self.exprs = []
        self.ids = {}
        # funcdecls, sorts
    def add(self, expr : smt.ExprRef) -> int:
        assert isinstance(expr, z3.ExprRef)
        z3_id = expr.get_id()
        eid = self.ids.get(z3_id)
        if z3_id is None:
            eid = len(self.exprs)
            z3.Z3_inc_ref(self.ctx, expr.as_ast())
            self.exprs.append(expr)
            self.ids[z3_id] = eid
            return eid
        else:
            return eid
    def __del__(self):
        for z3_id in self.exprs:
            z3.Z3_dec_ref(self.ctx, z3_id)
    def get_id(self, eid : int) -> int:
        return self.exprs[eid]
    def eq(self, a : int, b : int) -> bool:
        return a == b
    def mk_true(self):
        t = z3.Z3_mk_true(self.ctx)
        z3_id = z3.Z3_get_id(t)
        if z3_id in self.ids:
            return self.ids[z3_id]
        else:
            eid = len(self.exprs)
            z3.Z3_inc_ref(self.ctx, t)
            self.exprs.append(t)
            self.ids[z3_id] = eid
            return eid

class Arena():
    def __init__(self):
        # make a new context
        self.ctx = z3.Context()
    def add(self, expr : smt.ExprRef) -> int:
        e = expr.translate(self.ctx)
        self.exprs
        


# Linear knuckledragger
Graham:
Yep! https://www.math.tau.ac.il/~aa/articles/linear_tcs.pdf

tldr; it's most straightforward if you focus on just proofs of theorems - no assumptions allowed. Then the main modification you need is that each line except the last needs to be used exactly once. Your axioms for -> are reflexivity, transitivity, and  (A->(B->C)) ->(B->(A->C)) (these are the combinators for a combinatory linear lambda calculus I think). Add double negation elimination and transposition, and define the multiplicative conjunction as ~(A->~B) to get the multiplicative fragment. For the additive connectives, you need and-introduction as a rule, not as an axiom, and you need an unusual axiom ((A->B)/\(A->C))->(A->B/\C). That's basically because additive and-introduction fuses the assumptions used to prove the left and right half of the conjunction, so in a Hilbert system you either need to introduce a conjunction using two statements that have been proven without any assumptions (that'd be the rule form, if you're working in a context with no assumptions), or via that unusual axiom.

philzook — 9/14/25, 4:42 PM
Interesting. My secret wondering was if somehow I could use python inference rule functions that del the incoming judgements to somehow implement a linear logic. Knuckledragger is a hilbert style system.  Somehow using meta linearity to impose object linearity. Something like this
def linear_modus(A : Proof, AlolliB : Proof):
  assert is_proof(A) and is_proof(AlolliB)
  assert is_lolli(AlolliB.thm) and AlloliB.thm.arg(0).eq(A.thm)
  B = AlolliB.thm.arg(1)
  del A
  del AlolliB
  return Proof(B)


# refined python


Ramana:
@check
def foo(n: int, m: 'p : int | p > 0') -> 'z : int | z > n':
    return m + n
Refinement types in Python 
@check raises an exception if the function fails to typecheck
https://gist.github.com/rnagasam/457d0000c07dbc4b441671cfdaa14c80


We're always circling back to my stringly typed language idea

What’s stringly typed? All types are strings? Is this a serious idea?
Whats the type of type? “Str”?
“Type”
Sounds kind of like linking a quotation hierarchy to a type hierarchy
Which kind of happens in that tarski universe thing https://ncatlab.org/nlab/show/Tarski+universe
Dis association — 9/12/25, 2:57 PM
It's a SIGBOVIK paper idea, so definitely "serious"
And types are regular expressions but with regex "vars" for function types

In [2]:
def vadd(n : int, u : "Vec(n)", v : "Vec(n)") -> "Vec(n)":
    match u, v:
        case Cons(x,xs), Cons(y,ys): return Cons(x + y, vadd(n-1, xs, ys))
        case Nil(), Nil(): return Nil()

# Countermodel search
Something Sequenty? A prover and counterprover. Something intuitionistic?

|= phi  == forall M, M |= phi
!|= phi == Not(forall M, M |= phi)  == Exists M, Not(M |= phi) == Exists M, M |= Not(phi)
|= Not(phi)

It's kind of modal logic feeling over models? Is that an insane thing to say?

M |= phi
M |= Not(phi)

!|= phi <-> M |= Not(phi)

It is not the case that exactly one of |= phi or |= Not(phi) hold.

Gam |= phi
BigGam |= phi

by |= phi
all |= phi


# Primtive recursion
guardfed recursion?
Lexicographic recursion?

https://en.wikipedia.org/wiki/Primitive_recursive_function "pure recursion" access to parameters and subcalls but not item itself.
The analog of a fold.

define_fold

Attach a motive? 

define_rec(name, vs, motive, **cases):

https://bentnib.org/posts/2011-04-28-folds-and-induction.html

Hmm. Subsume induction?
https://people.cs.nott.ac.uk/pszgmh/fold.pdf ackermann is primtiive recursive if we have higher order fold

https://stackoverflow.com/questions/30419419/what-is-the-difference-between-primrec-and-fun-in-isabelle-hol
https://isabelle.in.tum.de/doc/functions.pdf

In [ ]:

def is_accessor(e, l):
    if e.eq(l):
        return True
    else:
        is_accessor()

def define_fix()
    # check recusrive calls are only accessor chains.
    # guarded recursion.

In [ ]:
from kdrag.all import *
import kdrag.theories.nat as nat
def define_primrec(fun_name, vs : list[smt.ExprRef], res_type : smt.SortRef, **cases):
    l = vs[0]
    assert isinstance(l, smt.DatatypeRef)
    params = vs[1:]
    sort = l.sort()
    f = smt.Function(fun_name, *[v.sort() for v in vs], res_type)
    acc = None
    for i in range(sort.num_constructors()):
        constr = sort.constructor(i)
        name = constr.name()
        args = [sort.accessor(i, j)(l) if constr.domain(j) != sort else f(sort.accessor(i, j)(l), *params) for j in range(constr.arity())]
        if acc is None:
            acc = cases[name](*args)
        else:
            acc = smt.If(sort.recognizer(i)(l), cases[name](*args), acc)
        del cases[name]
    if len(cases) != 0:
        raise ValueError("unused cases", cases)
    return kd.define(fun_name, vs, acc)

n = smt.Const("n", nat.Nat)
id_ = define_primrec("id", [n], nat.Nat, S=lambda pred: nat.Nat.S(pred), Z=lambda: nat.Nat.Z)
id_.defn
m = smt.Const("m", nat.Nat)
plus = define_primrec("plus", [n, m], nat.Nat, S=lambda psum: nat.Nat.S(psum), Z=lambda: m)
plus.defn

       id(n) ==
       If(is(S, n), S(id(pred(n))), If(is(Z, n), Z, c!102))) to ForAll(n, id(n) == If(is(S, n), S(id(pred(n))), Z))
       plus(n, m) ==
       If(is(S, n),
          S(plus(pred(n), m)),
          If(is(Z, n), m, c!103))) to ForAll([n, m],
       plus(n, m) == If(is(S, n), S(plus(pred(n), m)), m))


|= ForAll([n, m],
       plus(n, m) == If(is(S, n), S(plus(pred(n), m)), m))

# Modules

Maude obj
https://www.mimuw.edu.pl/~chrzaszcz/papers/Chrzaszcz-Jouannaud_From-OBJ-to-ML-to-Coq.pdf From OBJ to ML to Coq

https://cafeobj.org/
maude

Ramana days PVS is moduly. Does PVS not have polymorphism?

Yeah, maybe the mayude family is subsort oriented but nevertheless not polymorphic.

My notes on egglog modules

https://arxiv.org/abs/2101.02690 Theorem Proving and Algebra
https://cseweb.ucsd.edu/~goguen/pubs/asbk.html Algebraic Semantics of Imperative Programs
Sheaves and Structures of Transition Systems  https://link.springer.com/chapter/10.1007/11780274_21
https://web.archive.org/web/20160303201750/http://formal.cs.uiuc.edu/goguenFest/

```python
def define_rec(name, l : smt.ExprRef, res_type : smt.SortRef, **cases):
    assert isinstance(l, smt.DatatypeRef)
    sort = l.sort()
    f = smt.Function(name, sort, res_type)
    acc = smt.FreshConst(res_type)
    for i in range(sort.num_constructors()):
        constr = sort.constructor(i)
        name = constr.name()
        args = [sort.accessor(i, j)(l) if constr.domain(j) != sort else f(sort.accessor(i, j)(l)) for k in constr.arity()]
        acc = smt.If(sort.recognizer(i)(l), cases[name](*args), acc)
        del cases[name]
    if len(cases) != 0:
        raise ValueError("unused cases", cases)
    return kd.define(name, [l], acc)

define_rec("add", Succ=lambda pred: smt.Lambda([y], ))
```

Open enums
open datatypes
A module with Int tag


Goguen
combining modules
Module of a ctagoery in goguen?

(SortRef, Lambda) pair
type SubSort = tuple[SortRef, ]


In [ ]:
class AutoM(Protocol):
    State : smt.SortRef
    Input : smt.SortRef
    Output : smt.SortRef
    start : smt.ExprRef
    f : smt.FuncDeclRef # Input State -> STate
    g : smt.FuncDeclRef # State -> Output

class Graph(Protocol):
    Edge : smt.SortRef
    Node : smt.SortRef
    src : smt.FuncDeclRef # Edge -> Node
    dst : smt.FuncDeclRef # Edge -> Node

class HomoMorphism(Protocol):
    

In [ ]:
class Sequence(Protocol): # protocol is like module signature
    T : smt.SortRef
    Elt : smt.SortRef
    length : smt.FuncDeclRef
    Nil : smt.ExprRef
    Cons : smt.FuncDeclRef
    hd : smt.FuncDeclRef
    tl : smt.FuncDeclRef
    nth : smt.FuncDeclRef
    mem : smt.FuncDeclRef
    rev : smt.FuncDeclRef
    append : smt.FuncDeclRef


#https://ocaml.org/manual/5.3/api/Set.S.html
class List(Sequence):
    def __init__(self, Elt):
        self.T = self.Datatype("List")
        self.Elt = Elt
        x,y,z = self.FreshVars("x y z", self.T)
        self.length = kd.define("length", [x], smt.If(x.is_nil, 0, 1 + self.length(x.tail)))
        self.is_empty = self.T.is_nil
        self.cons = self.T.Cons
        self.hd = self.T.hd
        self.tl = self.T.tl
        nth = smt.Function("nth", self.T, smt.IntSort(), self.Elt)
        self.nth = kd.define("nth", [x,n], smt.If(n <= 0, x.hd, nth(x.tl, n-1)))
        mem = smt.Function("mem", self.Elt, self.T, smt.BoolSort())
        self.mem = kd.define("mem", [y,x], smt.If(x.is_nil, False, smt.Or(x.hd == y, self.mem(y, x.tl))))
        self.rev = kd.define("rev", [x], smt.If(x.is_nil, x, self.rev(x.tail).cons(x.head)))
        self.append = kd.define("append", [x,y], smt.If(x.is_nil, y, self.append(x.tail, y).cons(x.head)))

        # self.length_nil = kd.prove(length(T.Nil) == 0, unfold=1)
        self.length_nil = self._length_nil()

    def _length_nil(self) -> kd.Proof:
        l = smt.Const("l", self.T)
        l = kd.Lemma(smt.ForAll([l], (self.length(l) == 0) == (l == self.Nil)))
        x = l.fix()
        l.induct(x)
        l.auto(unfold=1)
        print(l)
        l.auto(by=self.length.defn)
        return l.qed()

class LawfulList():
    def __init__(self, L: List):
        self.L = L
        x,y,z = L.FreshVars("x y z", L.T)
        self.append_nil = kd.prove(L.append(x, L.Nil) == x, unfold=1)
        self.append_assoc = kd.prove(L.append(L.append(x,y), z) == L.append(x, L.append(y,z)), unfold=2)
        self.length_append = kd.prove(L.length(L.append(x,y)) == L.length(x) + L.length(y), unfold=2)
        self.mem_append = kd.prove(smt.ForAll([L.Elt], smt.ForAll([x,y], L.mem(z, L.append(x,y)) == smt.Or(L.mem(z,x), L.mem(z,y)))), unfold=2)


class Set(Protocol):
    T : smt.SortRef
    Elt : smt.SortRef
    Empty : smt.ExprRef
    Full : smt.ExprRef

class Nat(Protocol):
    subset : smt.QuantifierRef # subsort? Something. Maude has subsort looking stuff



In [35]:
import functools
from dataclasses import dataclass
class Foo():
    table = {}
    @functools.cache
    def __new__(cls, x):
        print("new")
        self = super().__new__(cls)
        self.x = x
        return self

x = Foo(1)
y = Foo(1)
assert x is y
z = Foo(2)
x is z

new
new


False

# smt eggbench
Random scramblnig / walk to generate big terms?

Do set equations also.






In [10]:
import kdrag.theories.bitvec as bv
from kdrag.all import *
for k,v in bv.BV8.__dict__.items():
    if kd.kernel.is_proof(v):
        print("(assert " + v.thm.sexpr() + ")")


(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8))) (= (bvadd x y) (bvadd y x))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8)) (z (_ BitVec 8)))
  (= (bvadd x y z) (bvadd x y z))))
(assert (forall ((x (_ BitVec 8))) (= (bvadd x #x00) x)))
(assert (forall ((x (_ BitVec 8))) (= #x00 (bvadd x (bvneg x)))))
(assert (forall ((x (_ BitVec 8))) (= #x00 (bvsub x x))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8))) (= (bvsub x y) (bvadd x (bvneg y)))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8))) (= (bvmul x y) (bvmul y x))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8)) (z (_ BitVec 8)))
  (= (bvmul x y z) (bvmul x y z))))
(assert (forall ((x (_ BitVec 8))) (= (bvmul x #x01) x)))
(assert (forall ((x (_ BitVec 8))) (= #x00 (bvmul x #x00))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8))) (= (bvand x y) (bvand y x))))
(assert (forall ((x (_ BitVec 8)) (y (_ BitVec 8)) (z (_ BitVec 8)))
  (= (bvand x y z) (bvand x y z))))
(assert (forall ((x (_ BitVec 8))) (= (bvand x #

# transfer functions


# Transition relations
I should do more with transition relations
transitive closure

prove verilog counter example.







In [ ]:
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
kd.define("trans_close", [n, R], smt.If(n.is_Z, smt.Lambda([x,z], x == z), smt.Lambda([x,z], smt.Exists([y], R[x,y], trans_close(n.pred, R)[y,z])))

# cffi speed

In [3]:
import z3
x = z3.Int("x")
x.serialize()
y = z3.Real("y")
y.serialize()

'(declare-fun F (Real) Bool)\n(declare-fun y () Real)\n(assert (F y))\n'

In [1]:
import z3

x = z3.Int("x")

In [3]:
%%timeit
x.get_id()

1.13 μs ± 210 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [46]:
os.path.join(os.path.dirname(z3.__file__), "include")

'/home/philip/philzook58.github.io/.venv/lib/python3.12/site-packages/z3/include'

In [6]:
from cffi import FFI
ffibuilder = FFI()

import os
header_path = os.path.join(os.path.dirname(z3.__file__), "include") #, "z3_api.h")
so_path = os.path.join(os.path.dirname(z3.__file__), "lib") #, "libz3.so") # there are actually two files?

# cdef() expects a single string declaring the C types, functions and
# globals needed to use the shared object. It must be in valid C syntax.
ffibuilder.cdef("""
    unsigned Z3_get_ast_id(void * c, void * t);
    void * Z3_mk_eq(void * c, void * a1, void * a2);

""")

# set_source() gives the name of the python extension module to
# produce, and some C source code as a string.  This C code needs
# to make the declared functions, types and globals available,
# so it is often just the "#include".
ffibuilder.set_source("_z3_cffi",
"""
     #include "z3.h"   // the C header of the library
""",
     libraries=['z3'], library_dirs=[so_path], include_dirs=[header_path])   # library name, for the linker

ffibuilder.compile(verbose=True)

generating ./_z3_cffi.c
the current directory is '/home/philip/philzook58.github.io/_drafts'


'/home/philip/philzook58.github.io/_drafts/_z3_cffi.cpython-312-x86_64-linux-gnu.so'

In [6]:
x.ctx_ref(), x.as_ast()
x.ctx_ref().value

809229680

In [ ]:
dir(x)


In [3]:
from _z3_cffi import ffi, lib
ctx = ffi.cast("void *", x.ctx_ref().value)
lib.Z3_get_ast_id(ctx, ffi.cast("void *", x.as_ast().value))

5

In [8]:
import sys
import os
print(open("/proc/self/maps").read())

00400000-00420000 r--p 00000000 103:05 87031863                          /usr/bin/python3.12
00420000-00703000 r-xp 00020000 103:05 87031863                          /usr/bin/python3.12
00703000-00a28000 r--p 00303000 103:05 87031863                          /usr/bin/python3.12
00a28000-00a29000 r--p 00627000 103:05 87031863                          /usr/bin/python3.12
00a29000-00ba7000 rw-p 00628000 103:05 87031863                          /usr/bin/python3.12
00ba7000-00bab000 rw-p 00000000 00:00 0 
30c54000-31e57000 rw-p 00000000 00:00 0                                  [heap]
7bed10000000-7bed10021000 rw-p 00000000 00:00 0 
7bed10021000-7bed14000000 ---p 00000000 00:00 0 
7bed18000000-7bed18043000 rw-p 00000000 00:00 0 
7bed18043000-7bed1c000000 ---p 00000000 00:00 0 
7bed1c000000-7bed1c029000 rw-p 00000000 00:00 0 
7bed1c029000-7bed20000000 ---p 00000000 00:00 0 
7bed229de000-7bed229df000 ---p 00000000 00:00 0 
7bed229df000-7bed231df000 rw-p 00000000 00:00 0 
7bed231df000-7bed24021

In [5]:
# precasting and remove lib. indirection
xptr = ffi.cast("void *", x.as_ast().value)
get_id = lib.Z3_get_ast_id
mk_eq = lib.Z3_mk_eq

In [43]:
%%timeit
get_id(ctx, xptr)

140 ns ± 2.35 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
%%timeit
mk_eq(ctx, xptr, xptr)

254 ns ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
ast1 = x.ast
ctx1 = x.ctx_ref()
z3.Z3_DEBUG = False

In [ ]:
%%timeit
z3.z3core.Z3_mk_eq(ctx1, ast1, ast1) 

752 ns ± 45.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


So maybe a fast zipper is worth it.

raw z3py - 30us, ctypes ~ 750ns, cffi ~250ns, c++ ~ 60ns

numba jit?
cython?
reverse ffi to lean?


In [ ]:
class Zipper():
    def __init__(self, e):
        self.ctx = e.ctx
        self.trail = []
        self.e0 = e # keep alive
        self.focus = e.as_ast
    def child(self, n):
        self.trail.append()
    def up(self):
        (decl, left, right) = self.trail.pop()
        children = left + [self.focus] + right
        self.focus = decl(self.ctx, *children)
    def done(self):
        while trail:
            self.up()
        return z3_ast_to_expr(self.ctx, self.focus)

    def __del__(self):
        for (decl, left, right) in self.trail:
            for ast in left:
                z3.z3core.Z3_dec_ref(self.ctx, ast)




SyntaxError: expected ':' (3970149753.py, line 10)

In [46]:
%%timeit
x.ctx

23.1 ns ± 0.549 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [53]:
%%file /tmp/testpypy.py
import z3
import timeit
def equality_check():
    c = z3.Context()
    x = z3.Int("x", ctx=c)
    return x == x

print(timeit.timeit(equality_check))

Writing /tmp/testpypy.py


In [67]:
%%file /tmp/test_z3.cpp

#include <z3++.h>
#include <benchmark/benchmark.h>

// Create some sample Z3 expressions
static void BM_Z3_EqualityCheck(benchmark::State& state) {
    z3::context c;
    z3::expr x = c.int_const("x");
    //z3::expr y = c.int_const("y");
    //z3::expr expr1 = x + y + 2;
    //z3::expr expr2 = y + x + 2; // Structurally different but equivalent

    for (auto _ : state) {
        // benchmark::DoNotOptimize(x.id());
        benchmark::DoNotOptimize(x == x);
    }

}

// Register the benchmark function
BENCHMARK(BM_Z3_EqualityCheck);

int main(int argc, char** argv) {
    ::benchmark::Initialize(&argc, argv);
    ::benchmark::RunSpecifiedBenchmarks();
    return 0;
}

Overwriting /tmp/test_z3.cpp


In [69]:
! g++ -O3 -march=native -o /tmp/test_z3 /tmp/test_z3.cpp -lz3 -lbenchmark && /tmp/test_z3

2025-08-25T10:32:24-04:00
Running /tmp/test_z3
Run on (16 X 5134 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x8)
  L1 Instruction 32 KiB (x8)
  L2 Unified 1024 KiB (x8)
  L3 Unified 16384 KiB (x1)
Load Average: 1.99, 1.07, 1.08
***WARNING*** CPU scaling is enabled, the benchmark real time measurements may be noisy and will incur extra overhead.
***WARNING*** Library was built as DEBUG. Timings may be affected.
--------------------------------------------------------------
Benchmark                    Time             CPU   Iterations
--------------------------------------------------------------
BM_Z3_EqualityCheck       62.6 ns         62.6 ns     10352946


In [126]:
%%file /tmp/test.rs
---
[profile.dev]
opt-level = 3
debug = false
split-debuginfo = '...'  # Platform-specific.
strip = "none"
debug-assertions = false
overflow-checks = false
lto = false
panic = 'unwind'
incremental = false
codegen-units = 16
rpath = false
[dependencies]
z3 = "0.16.1"
criterion = "0.7"
---

use criterion::{criterion_group, criterion_main, Criterion};
use z3::{ast::Int, Config, Context, ast::Ast};
use std::hint::black_box;

fn mk_eq(c : &mut Criterion) {
    let x = Int::new_const("x");

    c.bench_function("Z3: x == x (build equality AST)", |b| {
        b.iter(|| {
            // `_eq` builds a Z3 equality; `black_box` ~ DoNotOptimize
            let eq_ast : z3::ast::Bool = x._eq(&x);
            black_box(eq_ast);
        });
    });
}

fn fibonacci(n: u64) -> u64 {
    match n {
        0 => 1,
        1 => 1,
        n => fibonacci(n-1) + fibonacci(n-2),
    }
}

fn criterion_benchmark(c: &mut Criterion) {
    c.bench_function("fib 20", |b| b.iter(|| fibonacci(black_box(20))));
}

//criterion_group!(benches, criterion_benchmark);

criterion_group!(benches, mk_eq);
criterion_main!(benches);

Overwriting /tmp/test.rs


In [127]:
! cargo +nightly  -Zscript  /tmp/test.rs --bench

   Compiling test- v0.0.0 (/tmp/test.rs)
  --> test.rs:20:20
   |
20 | use z3::{ast::Int, Config, Context, ast::Ast};
   |                    ^^^^^^  ^^^^^^^
   |
   = note: `#[warn(unused_imports)]` on by default

  --> test.rs:35:4
   |
35 | fn fibonacci(n: u64) -> u64 {
   |    ^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

  --> test.rs:43:4
   |
43 | fn criterion_benchmark(c: &mut Criterion) {
   |    ^^^^^^^^^^^^^^^^^^^



    Finished `dev` profile [optimized] target(s) in 1.72s
     Running `/home/philip/.cargo/target/e8/a8b1c02d8deab1/debug/test- --bench`
Z3: x == x (build equality AST): Analyzingg 100 samples in estimated 5.0001 s (32M iterations)
                        time:   [151.30 ns 158.00 ns 166.03 ns]
                        change: [+563.68% +601.06% +640.98%] (p = 0.00 < 0.05)
                        Performance has regressed.
Found 14 outliers among 100 measurements (14.00%)
  4 (4.00%) high mild
  10 (10.00%) high severe



Ok, so more like a factor of 3-4
That's maybe not worth doing a whole thing on.

Arena pointers.
deref gc when the thing dies.



In [7]:
%%timeit
x == x

39.6 μs ± 3.19 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
x.as_ast()

In [ ]:
%%timeit
lib.Z3_get_ast_id(ctx, ffi.cast("void *", x.ast.value))

TypeError: initializer for ctype 'void *' must be a cdata pointer, not int

In [36]:
%%timeit
x.get_id()

1.2 μs ± 100 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
x.get_id()

5

# Path

pathmap for simp and others
unfold

Make a nice zipper?
folds, get, set

https://lean-lang.org/theorem_proving_in_lean4/The-Conversion-Tactic-Mode/ 
https://rocq-prover.org/doc/V8.9.1/refman/proof-engine/ssreflect-proof-language.html

In [ ]:
def pathmap(e, f, path : list[int]) -> smt.ExprRef:
    if path:
        args = e.children()
        args = args[:path[0]] + [pathmap(e.arg(path[0]), f, path[1:])] + args[path[0]+1:] 
        return e.decl()(*args)
    else:
        return f(e)

def getpath(e, path : list[int]) -> smt.ExprRef:
    for i in path:
        e = e.arg(i)
    return e

def setpath(e, t, path : list[int]) -> smt.ExprRef:
    return pathmap(e, lambda : t, path)

# Tele
Vec as subset of Functions.
Should subsort take in known sets of previous entries also?
x,Tx, y,Ty => Tf[f(x,y,z)]
Useful to synthesize more tpye info?

zero(n) vs zero
argmin(n,v : Vec n)

Vec n <= Vec m




In [ ]:
Arr = smt.DeclareSort("Arr")
Ob = smt.DeclareSort("Ob")
Hom = smt.Function("Hom", Ob, Ob, set_.Set(Arr))
#family.register(Hom, Hom)
a,b,c,d = kd.FreshVars("a b c d", Ob)
f,g,h,k = kd.FreshVars("f g h k", Arr)
declare("comp", [a, b, c, (f, Hom(a,b)), (g, Hom(b,c))], Hom(a,c))
declare("id", [a], Hom(a,a))
kd.notation.matmul.register(Arr, comp)
axiom(tele.TForAll([a,b,(f, Hom(a,b))], f @ id(b) == f))
axiom(tele.TForAll([a,b,(f, Hom(a,b))], id(a) @ f == f))
axiom(tele.TForAll([a,b,c,d,(f, Hom(a,b)), (g, Hom(b,c)), (h, Hom(c,d))], 


class Category(Protocol):
    Hom : smt.FuncDeclRef
    comp : smt.FuncDeclRef
    id : smt.FuncDeclRef
    right_id : kd.Proof
    left_id : kd.Proof
    assoc : kd.Proof

class Functor(Protocol):
    C : Category
    D : Category
    F_ob : smt.FuncDeclRef
    F_hom : smt.FuncDeclRef
    preserves_id : kd.Proof
    preserves_comp : kd.Proof





In [ ]:
from kdrag.all import *
import kdrag.theories.real as real

RSeqSet = set_.Set(real.RSeq)

a,b,c = kd.FreshVars("a b c", real.RSeq)
m,n,i,j = kd.FreshVars("m n i j", smt.IntSort())
def Vec(n: int) -> kd.utils.FuncRef:
    return smt.Lambda([a], kd.QForAll([i], smt.Or(i < 0, i >= n), a[i] == 0) )

vec = kd.define("Vec", [n], Vec(n))


vec_mono = kd.prove(kd.QForAll([m,n], n <= m, tele.TForAll([(a, vec(n))], vec(m)[a])), unfold=1)

zero = tele.define("zero", [n], vec(n), smt.Lambda([i], smt.RealVal(0)), unfold=1)

zero(3)

def infer_type(e : smt.ExprRef) -> kd.utils.FuncRef: # ctx
    return tele._tsig[e.decl()](*e.children())

infer_type(zero(3))
tele.has_type([], zero(3), vec(4), by=[vec_mono])

#(n, tele.Pos)
add = tele.define("add", [n, (a, vec(n)), (b, vec(n))], vec(n), smt.Lambda([i], a[i] + b[i]), unfold=1)
infer_type(add(3, zero(3), zero(3)))




#dot = tele.define("dot", [n, (a, vec(n)), (b, vec(n))], smt.RealSort(), smt.Sum([i], smt.And(i >= 0, i < n), a[i] * b[i]), unfold=1)



       Vec(n!394) ==
       (Lambda(a!390,
               ForAll(i!395,
                      Implies(Or(i!395 < 0, i!395 >= n!394),
                              a!390[i!395] == 0))))) to ForAll(n!412,
       Vec(n!412) ==
       (Lambda(a!408,
               ForAll(i!413,
                      Implies(Or(i!413 < 0, i!413 >= n!412),
                              a!408[i!413] == 0)))))
       add(n!394, a!390, b!391) ==
       (Lambda(i!395, a!390[i!395] + b!391[i!395]))) to ForAll([n!412, a!408, b!409],
       add(n!412, a!408, b!409) ==
       (Lambda(i!413, a!408[i!413] + b!409[i!413])))
       sum(n!394, a!390) ==
       If(n!394 < 0,
          0,
          a!390[n!394] + sum(n!394 - 1, a!390))) to ForAll([n!412, a!408],
       sum(n!412, a!408) ==
       If(n!412 <= 0,
          0,
          a!408[n!412 - 1] + sum(n!412 - 1, a!408)))


2

In [ ]:
import kdrag.theories.int as int_

sum = smt.Function("sum", smt.IntSort(), real.RSeq, smt.RealSort())
sum = kd.define("sum", [n, a], smt.If(n <= 0, 0, a[n-1] + sum(n-1,a)))
rw.full_simp(sum(2, smt.K(smt.IntSort(), smt.RealVal(1))))

x = smt.Real("x")
l = kd.Lemma(kd.QForAll([n,x], n >= 0, sum(n, smt.K(smt.IntSort(), x)) == n * x))
n,x = l.fixes()

"""
l.induct(n)
l.auto()
l.auto(unfold=1)
n1 = l.fix()
l.unfold(sum)
l.simp()
"""
l.intros()
l.induct(n, using=int_.induct_nat)
l.auto(unfold=1)
n = l.fix()
l.intros()
#l.intros()
l.unfold(sum)
l.simp()
l.auto()
l.auto()
l.qed()


argmin = smt.Function("argmin", smt.IntSort(), real.RSeq, smt.IntSort())
argmin = kd.define("argmin", [n, a], smt.If(n <= 0, 0, smt.If(a[n-1] < a[argmin(n-1,a)], n-1, argmin(n-1,a))))

l = kd.Lemma(kd.QForAll([n, x], n >= 0, argmin(n, smt.K(smt.IntSort(), x)) == 0))
argmin = 


       sum(n!628, a!408) ==
       If(n!628 <= 0,
          0,
          a!408[n!628 - 1] + sum(n!628 - 1, a!408))) to ForAll([n!633, a!408],
       sum(n!633, a!408) ==
       If(n!633 <= 0,
          0,
          a!408[n!633 - 1] + sum(n!633 - 1, a!408)))
       argmin(n!633, a!408) ==
       If(n!633 <= 0,
          0,
          If(a!408[n!633 - 1] <
             a!408[argmin(n!633 - 1, a!408)],
             n!633 - 1,
             argmin(n!633 - 1, a!408)))) to ForAll([n!638, a!408],
       argmin(n!638, a!408) ==
       If(n!638 <= 0,
          0,
          If(a!408[n!638 - 1] <
             a!408[argmin(n!638 - 1, a!408)],
             n!638 - 1,
             argmin(n!638 - 1, a!408))))


# Knuckle core
prove
define unfold
generalize, freshvar, instan
exists_intro, exists extro


# temp def

Like `consider`

__enter__ and __exit__ to clear out.
Gide more from the solver. Useful inside lemma?




# Jules jacobs macros vs internal

https://www.reddit.com/r/ProgrammingLanguages/comments/1aigns2/discussing_isabellehol_a_proof_assistant_for/kova7l5/

"This is a good question.

The advantage of higher-order logic over first-order logic is that propositions (and predicates) become first-class objects that you can abstract over in the theory.

Similarly, the advantage of type theory over higher-order logic is that types (and type constructors) become first-class objects that you can abstract over in the theory.

In that sense, if you understand what higher order logic gives you over first order logic, then you can understand what type theory gives you over higher order logic by analogy.

In both cases you can work around these limitations to some extent by using the meta-language as a macro system. For instance, if you only had first-order logic but you wanted to write a lemma that says something about a predicate (say, the strong induction lemma for natural numbers that is generic over some predicate P : nat -> prop), you could write the lemma as a macro that takes as input a predicate represented as a meta-level function that produces a formula, and write the proof of the lemma as a macro that generates a new proof for each new predicate P that the lemma is used on.

Similarly, in HOL you cannot write down code that is generic over a type constructor, e.g. code generic over a monad. Nor can you write down lemmas about such generic code. You can write a meta-level macro that does it, similar to how you can work around the limitations of first-order logic regarding propositions and predicates.

The big problem with this workaround is that now all your code and lemmas are checked at the use site instead of the definition site, similar to how C++ templates work. This also means that you can never write down and prove such a generic theorem; you can only generate and check instances of it.

The other main way to work around these problems is to not work in first order logic, but instead work in some theory built on top of first order logic. That's what ZFC set theory does. You can do something analogous in HOL, by not really working with types at all, but rather work in some sufficiently general un(i)typed universe and always work with predicates/subsets of that."



In [11]:
# diff

In [1]:
from kdrag.all import *

smt.Tuple(smt.IntSort(), smt.IntSort())

AttributeError: module 'kdrag.smt' has no attribute 'Tuple'

# Schema Hypotheses
I could make a freshvar inside a hypothesis.

Given Exists([x], P(x)) -->  y, P(y) fresh_hypvar
We get to associate a set to y by assumption

|- Q(y)    y,P(y) fresh
----------------------------
      forall y, P(y) => Q(y)


This is telescoped version of FreshVar
TFreshVar



BitVec(N)






In [ ]:
import kdrag.contrib.telescopes as tele
import kdrag.theories.set as set_



ModuleNotFoundError: No module named 'kdrag.contrib.telescopes'

`|- exists x, T(x) => T(yfresh)`

`|- exists x, T(x) == T(yfresh)` ? I mean yea, but I don't know that this helps.

p : 1 == 2

In [ ]:
@dataclass(frozen=True)
class FreshEvidence():
    v : smt.ExprRef
    hyp : smt.ExprRef

def TFreshVar(prefix : str, subsort : smt.ArrayRef):
    v = smt.FreshConst(subsort.domain(), prefix=prefix)
    ev = FreshEvidence(v, subsort[v])
    v.fresh = ev
    return v, kd.axiom(smt.Exists([v], subsort[v]) == subsort[v], by=["tfreshvar"])
# Or pass in 


def generalize(v, e : kd.Proof):
    assert isfresh(v) and is_proof(e)
    return kd.axiom(smt.ForAll([v], v.hyp, e))



# Lean isabelle call
IQ? MCP servers?
I shouldn't be wrapping this stuff.
PyPantograph?

IQ 

https://github.com/stanford-centaur/PyPantograph

In [4]:
from pantograph import Server
from pantograph.expr import Site, TacticHave, TacticExpr, TacticMode

server = await Server.create()
state0 = await server.goal_start_async("forall (p q: Prop), Or p q -> Or q p")
state0

GoalState(#0, goals=[Goal(id=None, variables=[], target='forall (p q: Prop), Or p q -> Or q p', sibling_dep=None, name=None, mode=<TacticMode.TACTIC: 1>)], _sentinel=#0

In [ ]:
import io
class LeanSolver(BaseSolver):
    def __init__(self):
        self.imports = []
        self.tactic = "grind"  # can replace by bv_decide or even tactic script. The string is just spliced in.\
        # Lean version needs to be explicitly picked or else elan will check for version which is very slow.
        self.lean_version = "4.22.0-rc3"
        self.server = await pantograph.Server.create()
        super().__init__()

    def check(self):
        with io.StringIO() as f:
            for imp in self.imports:
                f.write(f"import {imp}\n")
            f.write("set_option linter.unusedVariables false\n")  # To remove an warning
            f.write("set_option grind.warning false\n")
            predefined = set()
            # make inductive datatype definitions
            for sort in collect_sorts(
                self.adds + [thm for thm, name in self.assert_tracks]
            ):
                if isinstance(sort, smt.DatatypeSortRef):
                    f.write(lean.of_datatype(sort))
                    f.write("open " + sort.name() + "\n")
                    for n in range(sort.num_constructors()):
                        cons = sort.constructor(n)
                        predefined.add(cons)
                        for i in range(cons.arity()):
                            f.write(lean.accessor_def(sort, n, i))
                            predefined.add(sort.accessor(n, i))
                elif sort.name() not in self.predefined_sorts:
                    f.write(lean.sort_axiom(sort))
            # state axioms for all non predefined declarations
            for decl in collect_decls(
                self.adds + [thm for thm, name in self.assert_tracks]
            ):
                if decl not in predefined and decl.name() not in predefined_names:
                    f.write(lean.decl_axiom(decl))
            # Make the actual goal
            f.write("theorem mythm : Not (True ")
            for expr in self.adds + [thm for thm, name in self.assert_tracks]:
                f.write("  /\\ \n")
                f.write(lean.of_expr(expr))
            f.write(f") := by {self.tactic}\n")
            f.write("#check mythm\n")
            f.flush()
        result = subprocess.run(
            ["elan", "run", self.lean_version, "lean", "--json", "/tmp/test.lean"],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            return smt.unknown
        else:
            result = json.loads(result.stdout)
            assert result["severity"] == "information"
            assert "mythm" in result["data"]
            return smt.unsat


https://github.com/ufmg-smite/lean-smt/blob/main/Test/Bool/Assoc.expected

In [ ]:
import z3

z3.parse_smt2_string("""
(set-logic ALL)
(declare-const q Bool)
(declare-const r Bool)
(declare-fun f (Bool Bool) Bool)
(declare-const p Bool)
(assert (not (= (= (f p (f q r)) (f (f p q) r)) true)))
(check-sat)
""")


file = """
theorem addition (p q : Bool) : p → p || q = dummy := by
  simp_all
  smt_show
"""


subprocess.run(["lean"])


SyntaxError: incomplete input (2838448339.py, line 14)

# Protocols

https://leanprover-community.github.io/mathlib4_docs/Init/Core.html#Std.Associative

In [ ]:
from typing import Protocol, runtime_checkable
from kdrag.all import *

type SetRef = FuncRef

@runtime_checkable
class Assoc(Protocol):
    S : SetRef
    f : smt.FuncDeclRef
    assoc : kd.Proof
@runtime_checkable
class Comm(Protocol):
    f : smt.FuncDeclRef
    comm : kd.Proof
@runtime_checkable
class Idem(Protocol):
    f : smt.FuncDeclRef
    idem : kd.Proof
@runtime_checkable
class LeftIdentity(Protocol):
    f : smt.FuncDeclRef
    left_id : kd.Proof
@runtime_checkable
class RightIdentity(Protocol):
    f : smt.FuncDeclRef
    right_id : kd.Proof
@runtime_checkable
class Identity(LeftIdentity, RightIdentity, Protocol): ...

@runtime_checkable
class Refl(Protocol):
    """
    forall x, rel(x, x) 
    """
    rel : smt.FuncDeclRef
    refl : kd.Proof

@runtime_checkable
class Antisymm(Protocol):
    """
    forall x y, rel(x, y) and rel(y, x) implies x = y
    """
    rel : smt.FuncDeclRef
    antisymm : kd.Proof

@runtime_checkable
class Asymm(Protocol):
    rel : smt.FuncDeclRef
    asymm : kd.Proof

@runtime_checkable
class Total(Protocol):
    rel : smt.FuncDeclRef
    total : kd.Proof

@runtime_checkable
class Irrefl(Protocol):
    rel : smt.FuncDeclRef
    irrefl : kd.Proof

class Or(Comm, Assoc, Identity):
    _a, _b, _c = smt.Bools("a b c")
    f = smt.Or(_a, _b).decl()
    comm = kd.prove(smt.ForAll([_a, _b], smt.Or(_a, _b) == smt.Or(_b, _a)))
    assoc = kd.prove(smt.ForAll([_a, _b, _c], smt.Or(smt.Or(_a, _b), _c) == smt.Or(_a, smt.Or(_b, _c))))

#class And()
#Add
#Mul
# 

Do I _Need_ a S parameter?

# Implicits ?
How?
Macro


# generalized rewriting calc
https://rocq-prover.org/doc/V8.18.0/refman/addendum/generalized-rewriting.html
contextual / telescoped calc


No I need a cong close tactic.



In [ ]:

def gen_cong(terms : list[smt.ExprRef], eqs : list[kd.Proof]):
    ts = set()
    ts.extend(kd.utils.)
    while True:
        for t1 in ts:
            c1 = t1.children()
            for t2 in ts:
                c2 = t2.children()
                if all((a1,a2) in eqs for a1,a2 zip(c1,c2)):
                    






In [ ]:
import operator


class ParamRelation():
    rel : smt.FuncDeclRef
    refl : kd.Proof
    trans : kd.Proof

    refl : Optional[kd.Proof] = None # forall x, rel(x,x)
    symm : Optional[kd.Proof] = None # forall x y, rel(x,y) -> rel(y,x)
    trans : kd.Proof # forall x y z, rel(x,y) and rel(y,z) -> rel(x,z)

# diciontary from relation of interest -> funcdecl -> forall rel(a0,a1), rel(a1,a2), rel(a0,a2) -> rel(f(a0), f(a2))
cong_db : dict[Rel, dict[smt.FuncDeclRef, kd.Proof]]= {}
refl_db : 
symm_db 
trans

# pretty expnsive
def cong_tactic(rel, t1, t2, eqs):
    goal = t1 == t2
    if goal.get_id() in eqs:
        return eqs[goal]
    else:
        if t1.decl() == t2.decl():
           cong_tactic(rel,c1,c2,eqs) for c1,c2 in zip(t1.children(), t2.children()) 

    # zip down t1 t2 and apply cong
    # only stop if 


def rewrite_tactic(rel, t1, t2, rule):



class Calc2():
    ctx = None
    start : smt.ExprRef
    t : smt.ExprRef
    pf : kd.Proof
    def __init__(self, start, rel=operator.eq, ctx=None):
        self.start = start
        self.rel
        self.t = start
        self.pf = kd.kernel.refl(start)
    def eq(self, t2, by=[]):
        pf = kd.prove(self.t == t2, self.start == t, by=by)
        self.pf = kd.kernel.trans(self.pf, pf)
        self.t = t2
    def qed(self):
        return self.pf
    def gqed(self, *vs):
        return self.pf.forall(vs)

In [ ]:
import operator
class Calc2():
    start : smt.ExprRef
    t : smt.ExprRef
    pf : kd.Proof
    def __init__(self, start, rel=operator.eq, ctx=None):
        self.start = start
        self.rel
        self.t = start
        self.pf = kd.kernel.refl(start)
    def eq(self, t2, by=[]):
        pf = kd.prove(self.t == t2, self.start == t, by=by)
        self.pf = kd.kernel.trans(self.pf, pf)
        self.t = t2
    def qed(self):
        return self.pf
    def gqed(self, *vs):
        return self.pf.forall(vs)


# leanexpr

In [ ]:
class LeanExpr: ...

class BVar(LeanExpr):
    idx : int
class FVar(LeanExpr):
    name : str
class Mvar(LeanExpr):
    name : str
class Sort(LeanExpr):
    level : int
class Const(LeanExpr):
    name : str
    levels : List[int]
class App(LeanExpr):
    fn : LeanExpr
    arg : LeanExpr
class Lam(LeanExpr):
    name : str
    v : LeanExpr
    body : LeanExpr
    binderinfo : str
class ForallE(LeanExpr):
    name : str
    v : LeanExpr
    body : LeanExpr
    binderinfo : str
class LetE(LeanExpr):
    name : str
    v : LeanExpr
    body : LeanExpr
    binderinfo : str
class Lit(LeanExpr):
    value : Any
class Mdata(LeanExpr):
    expr : LeanExpr
    data : Any
class Proj(LeanExpr):
    name : str
    expr : LeanExpr
    idx : int


# datatypesort

There has always been a DatatypeSort for Seq (and Array?)?




# Better unfold
Make axiom schema that does substitute_fun
auto_unfold collects up decls.


The arguments of the decl fun aren't for granted.


unifying / rewriting modulo z3.simplify

# Lemma stack

If self.lemmas was held in a stack, we could push pop it to make smaller lemma contexts.
z3 might be a bit happier.

We're emulating a runtime stack manually. Why?

The return is allowed to be filled in by z3. How could a language runtime do this
We are able to tag the thing we want in the answer.
[l1,l2,l3, None,None], extra





Lemma
class GoalNode():
class LeafGoal(GoalNode)
class OpenGoal(lemmas, goals, cb):
https://github.com/aqjune/hol-light-materials/blob/main/TacticDetails.md

give fix vars user given names



# eexists


axiom forms of z3 operations. We do want reliable forms of everything.

def rewrite(, eqs):
    subst = eq.arg(0), eq.arg(1) 
    substitute()

def unfold(t, defns):
    substitute_funs(t, )






In [ ]:
def rename_vars(vs, t):
    assert isinstance(t, smt.QuantifierRef, smt.free_in(t,vs))
    vs1, body = open_binder(t)
    body = smt.substitute(body, zip(vs1,vs))
    if t.is_forall():
        t2 = smt.ForAll(vs, body)
    elif t.is_exists():
        t2 = smt.ForAll(vs, body)
    elif t.is_lambda():
        t2 = smt.Lambda(vs, body)
    else:
        raise ValueExecption()
    return t2, kd.axiom(t == t2, by=["rename", t, vs])

In [ ]:
def eexists(self):
    goal = self.top_goal()
    assert isinstance(goal.conc, smt.QuantifierRef) and goal.conc.is_eexists()
    vs, body = open_binder(goal.conc)
    self.uvars.add(vs)

    return vs

def solve_for(self, x): # They don't explicily have a solve for
    # assign to variable in context
    assert x in self.uvars

    # Do branchjign search, try sympy, try other



# Proof objects

"A format and standalone checker of Big Step SMTLIB proofs"



# HOl light
https://hol-light.github.io/tutorial.pdf
opam install.
They are making this more usable
https://hol-light.github.io/
https://github.com/aqjune/hol-light-materials/tree/main?tab=readme-ov-file

In [ ]:
%%script hol.sh
print_endline "hello world";;;

# Transitive Closure induction

This is an alternative to InductiveRel. Similar idea though.
We could be using an explicit Nat.

trans(r, n) = cond(
    n < 0, undefined,
    n == 0, r(a,b),
    n > 0, Exists([c], r(c,b), trans(r,n-1)(a,c)
) 


https://mathoverflow.net/questions/238188/can-epsilon-induction-be-derived-from-the-transitive-closure-operator#:~:text=The%20induction%20of%20the%20transitive,P(x%2Cy).
https://cs.stackexchange.com/questions/74449/induction-rules-for-reflexive-transitive-closure

In [ ]:
from kdrag.all import *



R = Function('R', A, A, B)
TC_R = TransitiveClosure(R)

# Induction over the the structure of the lcousre
def trans_induct(R):
    def transitive_schema(P,a,b):
        return smt.Implies( smt.And( R[x,y] => P[x,y],  smt.And(P[x,y], R[y,z]) => P[x,z], TransitiveClosure(R)(a,b))
                                                        P[a,b])



AttributeError: module 'kdrag.smt' has no attribute 'Transitive'

# Calc2



# Set ZF
Lamport was a proponent of.
Operators vs functions as class functions
 https://lamport.azurewebsites.net/pubs/lamport-types.pdf Should Your Specification Language Be Typed?


On a somewhat related note, when people say List or Stream are greatest or least fixed points, I don’t understand “in what?”
https://lamport.azurewebsites.net/pubs/lamport-types.pdf
Some interesting stuff in here
Is List a class function?
There’s something going on with parameters vs indices too
https://leanprover.github.io/functional_programming_in_lean/dependent-types/indices-parameters-universes.html
Parameters don’t need to bump universes, indices do
What is the set theoretic analog of these principles
I’m guessing bumping universes is very similar to class vs set distinction
https://en.wikipedia.org/wiki/Reflection_principle
Class vs set or universe bumping is a big concept that always titillates. I don’t know what I’d need to see to make it feel coherent.
I think List/Stream being gfp vs lfp is in some implicit set theoretic model that 90% of the people who say it don’t know it in an actually precise way.
Semantics of type theory
Given a set A in zfc, using what principles and encodings can I construct something List(A) or Stream(A)?
What unctions or class functions?
What if A is a class?
What if A is in some universe?
I know I can construct kuratowski ordered pairs. I can construct the naturals and even some ordinals

WRT the "in what" question - Yeah, it seems like usually classes or sets. It makes sense to talk about coalgebras over any category, and as long as it has products you can think about "the lists of A in that category", as the initial algebras of the functor X |-> A * X , if those exist. But at least some of the actual results need you to be more specific about the ambient category, like Aczel's theorem here: https://plato.stanford.edu/entries/nonwellfounded-set-theory/#4.5.

It is somewhat strange that this article is the one with a good conception of this topic


# Propositions as sets of proofs in python
Pick a notion of proof object.




In [ ]:
class Prop():
    formula: object
    proofs : set[object]

# or use python name layer to name my formula. Can't have structured names then?
type Proof = object
type Prop = set[Proof]
def is_prop(x):
    return isinstance(x, Prop)

a_equals_b = set()
is_prop(a_equals_b)

# is_prop(a.eq.b)  # more structured?  is_prop(a.eq(b))

# Proof irrelevance is a confusing axiom to ask.
# Souffle style choice is one possilbiity
class SingletonSet(set):
    def add(self, item):
        if len(self) == 0:
            super().add(item)





# Transparent define


If I used recfunction in reducible_define
How much does this infect everything though?



# schematic variables
schematic variables via global tracking

schematic vars are the analog of unify vars in prolog _answers_

realtion to metamath vars?

I don't love this makes the system a bit weirder

I have a new scam for the knuckledragger kernel.
I've dealt with proving foralls via a herband axiom schema. It's very awkward
I've considered making my proof object more like a sequent, carrying a list of variables to be considered schematic
But here's the new idea: Make a new judgement that x schematic
And then I can add a forall around a proven thing if all the variables I want to quantify over are also proven schematic
And the only way you introduce schematic is through calling FreshConst
Part of the issue is that I have to use z3, so I can't add a constructor to my expression ast
It's a funny shell game, but after 5 minutes of hot headed thinking I don't see how it breaks

Instead of explicitly using a context list in Proof to know my schema vars, I am using the _python_ context and SchemaVar judgement objects

Could I do the same with hypotheticals?
Implies(A, B)

hA = hypothetical(A)
Doesn't give back...

prove(yada, by=[])
all(isisntance(p, Proof) or isinstance(p, Hypothetical))
  hyps = []


Proof()
    hyps : set[smt.ExprRef]
    conc : smt.ExprRef

def implI(hyp, pf):
    assert hyp in pf.hyps
    axiom(smt.Implies(hyp, pf.conc), hyps=pf.hyps - {hyp})

But if i do this, why not

prove(thm ,by=[], schematic=vs)
    smt.prove(smt.ForAll(vs, smt.And(by), thm))
    Proof(vs, thm, reasons=by)

but this is no good? This is just...
Well, the vs float. That's the difference. I can use them in the by.

Is this really

https://en.wikipedia.org/wiki/Metavariable

https://philpapers.org/archive/CORSTC.pdf  SCHEMATA: THE CONCEPT OF SCHEMA IN THE HISTORY OF
LOGIC Corcoran

https://en.wikipedia.org/wiki/Tarski%27s_undefinability_theorem


Something I think is interesting about Knuckledragger is that the meta (python) is very different than the object language (HOL/SMTLIB). I think this makes the concepts easier to distinguish, whereas many systems try to make them look similar and easier to mistake. They manipulate math inside of math and distinguish meta from object by font if at all.
 ⊦ true("(> x 0)") == (x > 0)


Tag schema var evidence onto the variable. That's always a little more convenient.

A dual idea is to mark Exists in goals as unif_vars instead of tracking a context inside Lemma.
It is kind of deliciously direct

def is_unif_var(x):
    return has_attr(v, "is_unif")

def UnifVar(prefix, sort):
    v = smt.FreshConst(sort,prefix=prefix)
    v.is_unif = True

Uhh, but the unif property disappears if it comes out of a subterm. Ok nevermind.


Could make a direct schematic var subtitution rule

def substitute_schema(e : kd.Proof,  subst):
    assert all(is_schema(v) for v,t in subst)




In [ ]:

# hypothetical facts
# A ypothetical fact doesn't mean much of antyhing.

class Hyp():




In [9]:
from kdrag.all import *
true = smt.Function("true", smt.StringSort(), smt.BoolSort())
def tarski_schema(fm : smt.BoolRef) -> kd.Proof:
    return kd.axiom(true(smt.StringVal(fm.sexpr())) == fm, by=["tarski", fm])
x = smt.Int("x")
tarski_schema(x > 0)

|- true("(> x 0)") == (x > 0)

In [ ]:

@dataclass(frozen=True)
class SchemaVarEvidence():
    """
    Do not instantiate this class directly. Use schema_var. It should always be created with a fresh variable
    """
    v : smt.ExprRef

def schema_var(sort : smt.SortRef, prefix: Optional[str] = None) -> smt.ExprRef:
    v = smt.FreshConst(sort, prefix=prefix)
    return SchemaVarEvidence(v)

def SchemaVars(names : str, sort : smt.SortRef):
    svs = [schema_var(sort, prefix=n) for n in names.split()]
    vs = [sv.v for sv in svs]
    return vs, svs


In [ ]:
class SchematicVar(smt.ExprRef): ...
    # kind of a proof that the variable is schematic. Meh

@dataclass(frozen=True)
class SchematicVarProof():
    v : smt.ExprRef

# This is database style proof kernel, which I tried to avoid.
_schema_var_db = set()
def schematic_var(sort, prefix=None):
    v= smt.FreshConst(sort,prefix=prefix)
    _schema_var_db.add(v)
    return SchematicVarProof(v)

def forallI(vs, body : Proof):
    assert all(v in _schema_var_db for v in vs), "Not all variables are schematic"
    assert isinstance(body, Proof), "Body must be a Proof"
    # automatically drop all of vs from table?
    return axiom(smt.ForAll(vs, body.thm), reason=["forallI", vs, body])

def forallI2(svs : list[SchemticVarProof], body : Proof):
    assert all(isinstance(sv, SchematicVarProof) for sv in svs), "All variables must be SchematicVarProof"



def grab_schema_var(sort):
    # try and reuse existing schematic vasriable?
    pass


# To control size of _schema_var_db, we can forget variables
# It is nice thought to probable at the start of an external proof file list all schema_var ever used.
# it is sufficient to note them at forallI usages though. That's all they do.
def forget_schematic_var(v):
    del _schema_var_db[v]

def clear_schematic_vars():
    global _schema_var_db
    _schema_var_db = set()


def open_binder_schema(e):
    svs = [schematic_var(e.var_sort(n), e.var_name(n)]
    return svs, e(*[sv.v for sv in svs])

# direct instantiation of a schematic cariable might be inconsistent. Maybe needs to be done globally? What about seeing through definitions?
# If I amke a definition that holds a schematic var? foo : Bool = x == ?a, Well I don't think I'll be able to trick it.
# But it's not obvious
#   

# foo(?x) == bar(?x)
#--------------------------
# lambda x foo(x) == lambda x bar(x)

def lambda_cong(svs, eq : kd.Proof):
    assert all(v.is_schematic() for v in vs), "All variables must be schematic"
    assert isinstance(eq, kd.Proof), "eq must be a Proof"
    assert eq.thm.is_eq(), "eq must be an equality"
    lhs,rhs = eq.children()
    axiom(Lambda([vs], lhs) == Lambda([vs], rhs), 
          reason=["lambda_cong", vs, eq])
    


def schematic_funcdecl(*sorts):
    smt.FreshFunction(*sort)

def schematic_sort(prefix=None):
    T = smt.FreshSort("schematic_sort")

def instan_sort(e : kd.Proof, A, T):
    assert A.is_schematic(), "A must be a schematic sort"
    assert not T.is_schematic(), "T must not be a schematic sort" # ? Maybe this is ok.




# Matching modulo definitions

Lean is rotten with this. Unification modulo execution or matching modulo definitions.
A persepctive on higher order unification?

whnf vs hnf


In [ ]:
def whnf(e):
    while True:
        if smt.is_app(e):
            if smt.is_select(e):
                f, *children = e.children()
                if isintance(f, smt.QuantifierRef) and f.is_lambda():
                # do subst
                e = f(*children)
            decl = e.decl()  
            elif e.decl() in kernel.defns:
                # do defn subst.
            else:
                # No more head reduction possible
                return e
        else:
            return e


## Lean
If i could bulk export to lean, I could get entire theories for free kind of.
Also if I can use mathlib
Linear algebra



# Graham's examples
https://github.com/gleachkr/lean-automation-showcase



# SBV examples

He's got some I don't have
https://hackage.haskell.org/package/sbv
Quicksort
Reverse
Sumreverse
binarysearch
majority
macarthy91



## Improveved lemma

In [1]:
from kdrag.all import *

def induct_nat(x, P):
    n = smt.FreshConst(smt.IntSort(), prefix="n")
    return kd.axiom(
        smt.Implies(
            smt.And(P(0), kd.QForAll([n], n >= 0, P(n), P(n + 1)), x >= 0),
            # ---------------------------------------------------
            P(x),
        )
    )


n, x = smt.Ints("n x")
P = smt.Array("P", smt.IntSort(), smt.BoolSort())
_l = kd.Lemma(
    kd.QForAll(
        [P],
        P(0),
        kd.QForAll([n], n >= 0, P(n), P(n + 1)),
        kd.QForAll([x], x >= 0, P(x)),
    ),
)
_P = _l.fix()
_l.intros()
_x = _l.fix()
_l.intros()
_l.induct(_x, using=induct_nat)
_l.auto()
_l.auto()
#_l.lemmas
_l.auto()
#induct_nat_ax = _l.qed()

ForAll(x, Implies(x >= 0, P!0[x]))
ForAll(P,
       Implies(And(P[0],
                   ForAll(n,
                          Implies(And(n >= 0, P[n]),
                                  P[n + 1]))),
               ForAll(x, Implies(x >= 0, P[x]))))


Nothing to do!

# Hoare



In [ ]:
ast.dump(ast.parse("x,y = 3,2; y = 7"))

"Module(body=[Assign(targets=[Tuple(elts=[Name(id='x', ctx=Store()), Name(id='y', ctx=Store())], ctx=Store())], value=Tuple(elts=[Constant(value=3), Constant(value=2)], ctx=Load())), Assign(targets=[Name(id='y', ctx=Store())], value=Constant(value=7))], type_ignores=[])"

In [ ]:



x,y,z = smt.Ints("x y z")
locals = {"x": x, "y": y, "z": z}
wp_str("x = 3; y = x + 2; z = y + 1", y > 0)



{3 + 2 > 0} x = 3; y = x + 2; z = y + 1 {y > 0}

# python dsl


In [ ]:
import ast
from kdrag.all import *
import inspect
def datatype(s : str, locals=None, globals=None) -> smt.DatatypeSortRef:
    if locals is None:
        locals = inspect.currentframe().f_back.f_locals
    if globals is None:
        globals = inspect.currentframe().f_back.f_globals
    mod = ast.parse(s)
    body = mod.body
    if len(body) != 1:
        raise ValueError(f"Expected a single type alias, got {ast.unparse(mod)}")
    type_alias = body[0]
    if not isinstance(body[0], ast.TypeAlias):
        raise ValueError(f"Expected a single type alias, got {ast.unparse(body[0])}")
    print(ast.dump(type_alias))
    dt = kd.Inductive(type_alias.name.id)
    todo = [type_alias.value]
    while todo:
        match todo.pop():
            case ast.BinOp(op=ast.BitOr(), left=left, right=right):
                todo.append(right)
                todo.append(left)
            case ast.Name(id=name):
                dt.declare(name)
            case ast.Call(func=ast.Name(id=name), args=args, keywords=[]):
                dt.declare(name, *[( f"{name}_{n}" ,locals[arg.id]) for n,arg in enumerate(args)]) # caller_frame = inspect.currentframe().f_back
            case ast.Call(func=ast.Name(id=name), args=[], keywords=keywords):
                dt.declare(name, *[(kw.arg ,locals[kw.value.id]) for kw in keywords]) # caller_frame = inspect.currentframe().f_back
            case _:
                raise ValueError(f"Unexpected subexpresison: {ast.unparse(type_alias.value)}")
    return dt.create()
    




Int = smt.IntSort()
Real = smt.RealSort()
Foo = datatype("type Foo = Biz | Bar | Baz(Int, Int) | Boz(x = Int, y = Int)")
# Foo = datatype("type Foo = Biz | Bar | Baz(Int, Int) | Boz(x = Int, y = {Int})") explicitly mark the things that should be interpolate in?
Foo.Biz
Foo.Baz.domain(1)
Foo.x.range()




TypeAlias(name=Name(id='Foo', ctx=Store()), type_params=[], value=BinOp(left=BinOp(left=BinOp(left=Name(id='Biz', ctx=Load()), op=BitOr(), right=Name(id='Bar', ctx=Load())), op=BitOr(), right=Call(func=Name(id='Baz', ctx=Load()), args=[Name(id='Int', ctx=Load()), Name(id='Int', ctx=Load())], keywords=[])), op=BitOr(), right=Call(func=Name(id='Boz', ctx=Load()), args=[], keywords=[keyword(arg='x', value=Name(id='Int', ctx=Load())), keyword(arg='y', value=Name(id='Int', ctx=Load()))])))


Int

In [ ]:
def setcomp(s:str, locals=None, globals=None): ...

ast.parse("{{x + 1 for x in A if x > 0}}")
#smt.Lambda([y], smt.Exists([x], y == x + 1))

In [ ]:
case ast.SetComp(elt=elt, generators=generators):
    res = rec(elt)
    y = smt.FreshConst(res.sort())
    genvars = []
    smt.Lambda([y], smt.Exists())
    xs = [ast.unparse(x) for x in elt.generators[0].iter]

In [ ]:
import ast

print(ast.dump(ast.parse("{x for x in Int for y in Int if x > 0 and y < 0}"), indent=4))




Module(
    body=[
        Expr(
            value=SetComp(
                elt=Name(id='x', ctx=Load()),
                generators=[
                    comprehension(
                        target=Name(id='x', ctx=Store()),
                        iter=Name(id='Int', ctx=Load()),
                        ifs=[],
                        is_async=0),
                    comprehension(
                        target=Name(id='y', ctx=Store()),
                        iter=Name(id='Int', ctx=Load()),
                        ifs=[
                            BoolOp(
                                op=And(),
                                values=[
                                    Compare(
                                        left=Name(id='x', ctx=Load()),
                                        ops=[
                                            Gt()],
                                        comparators=[
                                            Constant(value=0)]),
               

In [ ]:
from kdrag.reflect import expr
from kdrag.all import *
x = smt.Int("x")
expr("x + 1")
expr("{x for x in int if x > 0}")

ValueError: ('Could not interpret expression', '{x for x in int if x > 0}', "SetComp(elt=Name(id='x', ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Name(id='int', ctx=Load()), ifs=[Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0)])], is_async=0)])")

# unapply

What about a table to register unapply patterns



In [ ]:
unapply = {}

def add_unapply(t):
    
#def add_unapply(pat, t): ?

# recfunction translate

Maybe I should just dump the definitions into the main ctx. Why not?
Well, sometimes I want to hide them. I dunno.
Be an interesting experiment.

In [ ]:
smt.RecAddDefinition

In [ ]:
from kdrag.all import *
def define_rec(name, args, body):
    """
    Define a recursive function with the given name, arguments, and body.
    """
    f_not_rec = kd.define(name, args, body)
    f = smt.RecFunction(name, *[arg.sort() for arg in args], body.sort())
    smt.RecAddDefinition(f, args, smt.substitute_funs(body, (f_not_rec, f(*[smt.Var(n, f.domain(n)) for n in range(f.arity())]))))
    return f_not_rec #?
n,m = smt.Ints("n m")
add = smt.RecFunction("add", smt.IntSort(), smt.IntSort(), smt.IntSort())

add1 = define_rec("add", [n, m], smt.If(n <= 0, m, add(n - 1, m) + 1))

smt.simplify(add(3,4))

#smt.simplify(add1(3,4))


7

In [ ]:
import z3
z3.z3core.Z3_to_func_decl

<function z3.z3core.Z3_to_func_decl(a0, a1, _elems=<z3.z3core.Elementaries object at 0x7d4458071cd0>)>

In [ ]:
from kdrag.all import *
import kdrag.theories.nat as nat

ctx = smt.Context()
z3.ZE_DEBUG = False

subst = []
for f, defn in kd.kernel.defns.items():
    #print(f)
    f = f.translate(ctx)
    f.ast = z3.z3core.Z3_to_func_decl(f.ctx.ref(), f.ast) #f.ast.translate(ctx)
    args = [arg.translate(ctx) for arg in defn.args]
    body = defn.body.translate(ctx)
    frec = smt.RecFunction(f.name(), *[f.domain(i) for i in range(f.arity())] ,f.range())
    #print(type(f), type(body))
    subst.append((f, frec(*[smt.Var(f.arity() - n - 1, f.domain(n)) for n in range(f.arity())])))
    subst[0][0].as_func_decl()
    smt.substitute_funs(body, *subst)
print(subst)
for f, defn in kd.kernel.defns.items():
    #print(f)
    f = f.translate(ctx)
    print(defn.args)
    for arg in defn.args:
        print(arg)
        print(arg.translate(ctx))
    args = [arg.translate(ctx) for arg in defn.args]
    print(args)
    body = defn.body.translate(ctx)
    frec = smt.RecFunction(f.name(), *[f.domain(i) for i in range(f.arity())] ,f.range())
    print(frec, args, smt.substitute_funs(body, *subst))
    smt.RecAddDefinition(frec, args, smt.substitute_funs(body, *subst))


def simp(t):
    t = t.translate(ctx)
    print(t)
    return smt.simplify(smt.substitute_funs(t, *subst))



simp(nat.double(nat.Z, nat.Z))







[(even, even(Var(0))), (odd, odd(Var(0))), (safe_pred, safe_pred(Var(0))), (from_int, from_int(Var(0))), (to_int, Var(0)), (double, double(Var(0))), (add, add(Var(1), Var(0)))]
[Int]
Int
even
[even]
even [even] Exists(y, odd(even, 2*y))
[Int]
Int
even
[even]
odd [even] Exists(y, odd(even, 2*y + 1))
[n]
n
Select
[Select]
safe_pred [Select] If(is(Z, Select), x!1, pred(Select))
[a]
a
a
[a]
from_int [a] If(a <= 0, x!1, S(from_int(a - 1)))
[n]
n
n
[n]
to_int [n] If(is(Z, n), 0, 1 + (pred(n)))
[n]
n
n
[n]
double [n] If(is(Z, n), x!1, S(S(double(pred(n)))))
[x, y]
x
x
y
x
[x, x]
add [x, x] If(is(Z, x), x, S(add(x, pred(x))))


Z3Exception: b'index out of bounds'

# Extract Langs
C boehm GC
Rust via maturin + RC
Separate z3 context + recfunction
Java py4j https://www.py4j.org/



Maybe the rust version could lay some claim to
Both fast and correct version of code.
Output Kani conditions.


https://docs.rs/numpy/latest/numpy/

copilot inspired.
State loop?







## rust

What about kdrag-rs? Just do it all in rust. Learning rust details could be fun


Could macro the shit out of it I suppose?



In [ ]:
%%file /tmp/test.rs
---cargo
[dependencies]
z3 = "0.12.1"
lazy_static = "1.5.0"
---

use z3::ast::Ast;
/*

use std::sync::LazyLock;

lazy_static::lazy_static! {
    static ref Z3_CTX: z3::Context = z3::Context::new(&z3::Config::new());
}

static Z3_CTX: LazyLock<z3::Context> = LazyLock::new(|| {
    z3::Context::new(&z3::Config::new())
 });


fn pmatch<'ctx>(ctx : &'ctx z3::Context, vs : , fm : &'ctx z3::ast::Dynamic<'ctx>, pat : &'ctx z3::ast::Dynamic<'ctx>) -> Vec<> {
    let mut todo = vec![(fm, pat)];
    while todo{

    }
}
*/

#[derive(Debug)]
struct Proof<'ctx> {
    fm : &'ctx z3::ast::Bool<'ctx>,
}
fn prove<'ctx>(ctx : &'ctx z3::Context, fm : &'ctx z3::ast::Bool) -> Proof<'ctx> {
    let solver = z3::Solver::new(ctx);
    solver.assert(&fm.not());
    if solver.check() == z3::SatResult::Unsat {
        Proof { fm }
    } else {
        panic!("Unsatisfiable");
    }
}


fn main(){
    let ctx = z3::Context::new(&z3::Config::new());
    let solver = z3::Solver::new(&ctx);
    let x = z3::ast::Int::new_const(&ctx, "x");
    let y = z3::ast::Int::new_const(&ctx, "y");
    let z = z3::ast::Int::new_const(&ctx, "z");

    let _true = z3::ast::Bool::from_bool(&ctx, true);
    let p = prove(&ctx, &_true);
    println!("Proof: {:?}", p.fm);

    let _T = z3::Sort::uninterpreted(&ctx, z3::Symbol::from("T"));


    solver.assert(&x._eq(&y));
    solver.assert(&x._eq(&(y + z)));

    if solver.check() == z3::SatResult::Sat {
        println!("Satisfiable");
        println!("Model: {:?}", solver.get_model());
    } else {
        println!("Unsatisfiable");
    }

}

Overwriting /tmp/test.rs


In [ ]:
! cargo +nightly -Zscript /tmp/test.rs

   Compiling test- v0.0.0 (/tmp/test.rs)
  --> test.rs:47:9
   |
47 |     let _T = z3::Sort::uninterpreted(&ctx, z3::Symbol::from("T"));
   |         ^^ help: convert the identifier to snake case: `_t`
   |
   = note: `#[warn(non_snake_case)]` on by default

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.10s
     Running `/home/philip/.cargo/target/e8/a8b1c02d8deab1/debug/test-`
Proof: true
Satisfiable
Model: Some(z -> 0
x -> 0
y -> 0
)


In [ ]:
import maturin

ModuleNotFoundError: No module named 'maturin'

In [ ]:
class RustModule():
    def __init__(self, modname):
        self.modname = modname
        self.funcs = {}
        self.types = []

    def add_type(self, type_code):
        self.types.append(type_code)
    
    def add_function(self, fun_name, fun_code):
        """
        Add a function to the module with the given name and code.
        """
        if fun_name in self.funcs:
            raise ValueError(f"Function {fun_name} already exists in module {self.modname}.")
        self.funcs[fun_name] = fun_code


    def __str__(self):
            types = "\n".join("#[derive(FromPyObject, IntoPyObject)]\n" + type_code for type_code in self.types)
            fun_code = "\n".join(f"#[pyfunction]\n{fun_code}" for fun_code in self.funcs.values())
            add_funs = "\n".join(f"m.add_function(wrap_pyfunction!({fun_name}, m)?)?;" for fun_name in self.funcs.keys())
            return f"""\
use pyo3::prelude::*;
use pyo3::wrap_pyfunction;
use pyo3_ffi::c_str;

{types}

{fun_code}

#[pymodule]
fn {self.modname}(m: &Bound<'_, PyModule>) -> PyResult<()> {{
    {add_funs}
    Ok(())
}}
"""

M = RustModule("kdrag_smt")
M.add_function("add", "fn add(n: i32, m: i32) -> i32 {n + m}\n}")
M.add_type("struct MyType { field1: i32, field2: String }")
print(str(M))

use pyo3::prelude::*;
use pyo3::wrap_pyfunction;

#[derive(FromPyObject, IntoPyObject)]
struct MyType { field1: i32, field2: String }

#[pyfunction]
fn add(n: i32, m: i32) -> i32 {n + m}
}

#[pymodule]
fn kdrag_smt(m: &Bound<'_, PyModule>) -> PyResult<()> {
    m.add_function(wrap_pyfunction!(add, m)?)?;
    Ok(())
}



## wasm

Extract to wasm, run it./

Hmm. wasmtime has gc... That sweetens the pie a lot.

https://developer.mozilla.org/en-US/docs/WebAssembly
https://developer.mozilla.org/en-US/docs/WebAssembly/Guides/Understanding_the_text_format

https://github.com/WebAssembly/gc/blob/main/proposals/gc/Overview.md
https://github.com/bytecodealliance/wasmtime-py

https://tanishiking.github.io/posts/wasm-gc/

Codegen an ematcher. We used to talk about this. Or codegenning actions.

Hmm. But maybe one can hold external references to the host... so maybe structured objects could be dealt with that way. Using python.

Or pretty print, process call wasmtime cli. Read output. Could be worse. Gets me ready for the day wasm_gc is avaiable in the bindings.
wasmer?



In [ ]:
from wasmtime import Store, Module, Instance, Func, FuncType

store = Store()
module = Module(store.engine, """
  (module
    (func $hello (import "" "hello"))
    (func (export "run") (call $hello))
  )
""")

def say_hello():
    print("Hello from Python!")
hello = Func(store, FuncType([], []), say_hello)

instance = Instance(store, module, [hello])
run = instance.exports(store)["run"]
run(store)

Hello from Python!


In [ ]:
from wasmtime import Store, Module, Instance

store = Store()
module = Module(store.engine, """
  (module
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
""")
instance = Instance(store, module, [])
myadd = instance.exports(store)["myadd"]
myadd(store,1,2)

3

In [ ]:
%%file /tmp/test.wat

  (module
    (type $point (struct (field $x i64) (field $y i64)))
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
  

Writing /tmp/test.wat


In [ ]:
! wasmtime compile --wasm gc /tmp/test.wat -o /tmp/test.wasm

/bin/bash: line 1: wasmtime: command not found


In [ ]:
from wasmtime import Store, Module, Instance, Config, Engine
config = Config()
config.wasm_gc = True
engine = Engine(config)
store = Store(engine)
module = Module.deserialize_file(store.engine, "/tmp/test.wasm")

WasmtimeError: failed deserialization for: /tmp/test.wasm

Caused by:
    Module was compiled with support for WebAssembly feature `gc` but it is not enabled for the host

In [ ]:
from wasmtime import Store, Module, Instance, Config, Engine
config = Config()
config.wasm_gc = True
engine = Engine(config)
store = Store(engine)
module = Module(store.engine, """
  (module
    (type $point (struct (field $x i64) (field $y i64)))
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
""")
instance = Instance(store, module, [])
myadd = instance.exports(store)["myadd"]
myadd(store,1,2)

WasmtimeError: failed to parse WebAssembly module

Caused by:
    struct indexed types not supported without the gc feature (at offset 0xb)

In [ ]:
def struct_of_dt(d : DatatypeSortRef):
    for c in d:
        field = " ".join(f"(field ${f.name()} {f.sort()}" for f in c.fields())
        cons = f"(struct ${d}.{c.name()} "
    return f"""
    (rec
        
    )
    """




In [ ]:
def to_wasm_type(sort : smt.SortRef):
    if sort == smt.BitVec(32):
        return "i32"
    elif sort == smt.BitVec(64):
        return "i64"

def wasm_of_expr(e : smt.ExprRef):
    # recurse through
    if 

    decl = e.decl()
    if e.is_add()
    



def to_wasm(name, args, body : smt.ExprRef):
    """
    Convert a SMT expression to a WebAssembly function.
    """
    return f"""
(module
  (func ${name} (export {name}) {params} (result {to_wasm_type(body.sort())})
    
  )

    """

## C bump alloc
https://nullprogram.com/blog/2023/09/27/  Arena allocator tips and tricks
https://www.gnu.org/software/libc/manual/html_node/Obstacks.html obstacks?

Interesintg example. I want this thing. How can I make it as bomb proof as possible.
There are some discussions that make a type generic allocator is funky in waysy ou might noit expect.

1. compile with fanalyzer and Wall Wextra
2. ask chatgpt to critique
3. Tests
4. Sanitizers
5. CBMC? undefinedness interpreters 
6. Frama-C? VST?
7. Look at the assembly. is it meaningful?

Some suggestions of using mmap and guard pages. But will this be annoying to make cross OS

Some other things suggested: reset or checkpoint arenas

In [ ]:
%%file /tmp/arena.c
#include <stddef.h>
#include <stdlib.h>
#include <stdio.h>

typedef struct {
    void *data;
    size_t size;
    size_t capacity;
} arena_t;

void *arena_alloc(arena_t *arena, size_t size) {
    while (arena->size + size > arena->capacity) {
        arena->capacity = (arena->capacity == 0) ? size : arena->capacity * 2;
        arena->data = realloc(arena->data, arena->capacity);
    }
    void *ptr = (char *)arena->data + arena->size;
    arena->size += size;
    return ptr;
}

// arena_init instead?
int arena_init(arena_t *arena) {
    arena->data = malloc(256);
    if (!arena->data) {
        return -1;
    }
    arena->size = 0;
    arena->capacity = 256;
    return 0;
}
/*
arena_t *arena_create() {
    arena_t *arena = malloc(sizeof(arena_t));
    if (!arena) {
        return NULL; // Allocation failed
    }
    arena->data = malloc(256); // Initial capacity
    arena->size = 0;
    arena->capacity = 256;
    return arena;
}
*/

void *arena_deref(arena_t *arena, size_t offset) {
    return (char *)arena->data + offset;
}

void arena_destroy(arena_t *arena) {
    if (!arena) return;
    if (arena->data) {
        free(arena->data);
    }
}


int main(){
    arena_t arena;
    if (arena_init(&arena) != 0) {
        fprintf(stderr, "Failed to initialize arena\n");
        return 1;
    }
    int *x = arena_alloc(&arena, sizeof(int));
    *x = 42;
    printf("Allocated int: %d\n", *x);
    arena_destroy(&arena);
    return 0;
}




Overwriting /tmp/arena.c


In [ ]:
! gcc /tmp/arena.c -O2 -Wall -Wextra -fanalyzer -o /tmp/arena && /tmp/arena 

Allocated int: 42


In [ ]:

"""
typedef struct Arena


"""

# combinators

A full set of combinators

Proof terms that I translate?

lambda([P], P)


APL is master of the ocmbiunators. An APL knuckledragger would use a heavy combinator style.
Knuckledragger numpy?
vectorized proofs

https://en.wikipedia.org/wiki/Combinatory_logic
https://combinatorylogic.com/index.html


forall xs, smt.And() => smt.Or()   normal form
forall xs, smt.And() => conc   normal form

@ * + ~ << >> combinators for Proofs? Hook and etc.
Kind of a fun idea. ssreflect?



Harrison book has some

pluck And(xs) => P  ====>   x[n] => And(xs - x[n]) => P
insert   x => And(xs) => P  ====> And(xs + x + xs) => P
sort And(xs) => P ====> And(sorted(xs)) => P






In [ ]:


def cut(a, b):

In [ ]:
def curry(pf : kd.Proof) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    assert smt.is_implies(pf.thm)
    hyp, conc = pf.thm.children()
    assert smt.is_and(hyp)
    args = hyp.children()
    return kd.axiom(smt.Implies(smt.And(args[:-1]), smt.Implies(args[-1], conc)), by=["curry", pf])
a,b,c = smt.Bools("a b c")
curry(kd.prove(smt.Implies(smt.And(a, b, c), a)))

# curryn ?

def uncurry(pf : kd.Proof) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    assert smt.is_implies(pf.thm)
    hyp, conc = pf.thm.children()
    assert smt.is_implies(conc)
    hyp1, conc = conc.children() 
    return kd.axiom(smt.Implies(smt.And(hyp, hyp1), conc), by=["uncurry", pf])
uncurry(curry(kd.prove(smt.Implies(smt.And(a, b, c), a))))

# modus is apply





|- Implies(And(And(a, b), c), a)

Design  as decorator?

A little weird. Interesting looking patterns.

```
def Ex(vs, P):
   def res(cb):

def ExistsE(vs, P):
    


def ForAllI(vs, P):
    vs, ab = kd.kernel.herb(smt.ForAll([vs], P))

def Pi(vs, P):
    def res(cb):
        vs, ab = kd.kernel.herb(smt.ForAll(vs, P))
        return modus(ab, cb(*vs))
    return res

@Pi([n], foo(n))
def step1(goal, vs)


@fixes(smt.ForAll([n], foo(n)))
def foo(goal, n):
    @fixes
    def res(goal1, m):
        return kd.prove(...)
    return res
```

In [ ]:
from typing import Callable
from functools import partial
#@partial
def fixes(goal : smt.QuantifierRef, cb : Callable[[smt.ExprRef,...], kd.Proof]) -> kd.Proof:
    vs, ab = kd.kernel.herb(goal)
    a = cb(*vs) # ab.thm.arg(0) pass in?
    pf = kd.kernel.modus(ab, a)
    assert pf.thm.eq(goal)
    return pf

foo = smt.Function("foo", smt.IntSort(), smt.BoolSort())
n = smt.Int("n")
pf1 = kd.axiom(smt.ForAll([n], foo(n)))
fixes(smt.ForAll([n], foo(n)), lambda n: pf1(n))



|- ForAll(n, foo(n))

In [ ]:
def aprove(goal, instans):
    intros(goal, lambda goal1, *args: kd.prove(goal1, by=instans(*args)))

# BHK 
simp is definitional equality.
Model
ForAllI(lambda x: f)
Pi()
ForAllI([x,y,z], lambda x,y,z: kd.prove(    ))

def ForAllI(vs, prop):
    def res(f):
        vs, lemma = herb(ForAll(vs, prop))
        p = f(lemma.arg(0), *vs)
        return modus(lemma, p)
    return res
    #def res(goal, *vs):

class CtxProof():
    ctx : 
    thm : smt.
    pf : kd.Proof
class CtxExpr():


def prove(fm):
    if hasattr(fm, "kdrag_ctx"):
        return prove(smt.ForAll(smt.Implies(fm.kdrag_ctx, fm)))
    else:
        return prove(fm)




def ImplI(prop):
    def res(f):
        p = prop.arg(1)
        p.kdrag_ctx = prop.arg(0) + prop.kdrag_ctx
        f(p)

        f(CtxProof())
        ? = ?(prop)


@ForAllI([x,y,z])
def add_comm(x : int, y : int, z : int):
    return kd.prove()

def forall(f):
    inspect()
    kd.kernal.herb

@forall
def add_comm(x : int, y : int, z : int):

type Prop = BoolRef
def ind_eq(motive : Callable([ExprRef,ExprRef], BoolRef), a, b, p : kd.Proof):
    t1 = motive(a,a)
    assert p.thm.eq(t1)
    t2 = motive(a,b)

def ImplI()


contextmanager
generators for algerbaic effects

def prove(p):
    ctx = get_ctx()
    kd.prove(ctx => )

global? ctx

l = Lemma(forall )
with l.fix() as y:
    with z.intros() as ih1:



# protocol

Protocls as replacement for some uses of typeclasses

As replacement for ExprRef

metatheory.jl had some interface.

Lazy Expr
def to_smt(self) -> smt.ExprRef: ...



In [ ]:
class Ast(Protocol):
    def arg(self, i: int) -> 'Ast': ...
    def children(self) -> list['Ast']: ...
    def eq(self, other: 'Ast') -> bool: ...
class Sort(Ast, Protocol):
    def name(self) -> str: ...
class FuncDecl(Ast, Protocol):
    def name(self) -> str: ...
    def arity(self) -> int: ...
    def domain(self, i: int) -> 'Sort': ...
    def range(self) -> 'Sort': ...
class Expr(Ast, Protocol):
    def __eq__(self, other: 'Expr') -> "Expr": ...
    # all operations?
    # is_add: ... operations. Hmm.




In [ ]:
from typing import Protocol, runtime_checkable
from kdrag.all import *

@runtime_checkable
class Monoid(Protocol):
    add: smt.FuncDeclRef
    zero: smt.ExprRef
    assoc: kd.Proof # |- forall x, y, z: add(add(x, y), z) == add(x, add(y, z))
    lunit: kd.Proof # |- forall x: add(zero, x) == x
    runit: kd.Proof # |- forall x: add(x, zero) == x

@runtime_checkable
class Group(Monoid, Protocol):
    inv: smt.FuncDeclRef # inverse operation
    inv_lunit: kd.Proof # |- forall x: add(inv(x), x) == zero
    inv_runit: kd.Proof # |- forall x: add(x, inv(x)) == zero

# alternative to SortDispatch
ExprRef.__add__ = lambda x,y: x.sort().add(x,y)


@runtime_checkable
class Assoc(Protocol):
    assoc: kd.Proof


add = (x + y).decl()
add.assoc = kd.prove(smt.ForAll([x, y, z], (x + y) + z == x + (y + z)))
isinstance(add, Assoc)


In [ ]:
Lattice = smt.DeclareSort("Lattice")
meet = smt.Function("meet", Lattice, Lattice, Lattice)
kd.notation.and_.register(Lattice, meet)
join = smt.Function("join", Lattice, Lattice, Lattice)
kd.notation.or_.register(Lattice, join)
x,y,z = smt.Consts("x y z", Lattice)
meet_assoc = kd.axiom(smt.ForAll([x,y,z], x & (y & z) == (x & y) & z))
meet_comm = kd.axiom(smt.ForAll([x,y], x & y == y & x))
meet_idem = kd.axiom(smt.ForAll(x, x & x == x))
join_assoc = kd.axiom(smt.ForAll([x,y,z], x | (y | z) == (x | y) | z))
join_comm = kd.axiom(smt.ForAll([x,y], x | y == y | x))
join_idem = kd.axiom(smt.ForAll(x, x | x == x))

#kd.notation.le([x,y], x & y == x)


# complete
sup = smt.Function("sup", set_.SetSort(Lattice), Lattice)
inf = smt.Function("inf", set_.SetSort(Lattice), Lattice)


monotone = smt.Function("monotone", smt.ArraySort(Lattice, Lattice), smt.BoolSort())


# https://en.wikipedia.org/wiki/Knaster%E2%80%93Tarski_theorem
gfp = sup(smt.Lambda([x], x <= f(x)))
lfp = inf(smt.Lambda([x], f(x) <= x))






# Fixed point

Could spacer or BMC be a good 

Could I use this for a typeclass like system?

for subtypes?

from kdrag.all import *

fp = smt.FixedPoint()
fp.set(engine="bmc")



# user propagators
?
I have some ideas about forall x, inj(proj(x)) = x
User propagators good for that maybe? A simple application.
is_set(smt.ArraySort())





# ProofSet
A direct BHK interp.
We can destructure formulas at the metalevel. Akin to quote in some respects.
https://en.wikipedia.org/wiki/Brouwer%E2%80%93Heyting%E2%80%93Kolmogorov_interpretation

But I struggled to do a Dialectica?


In [ ]:
def ProofSet(p : smt.BoolRef) -> smt.QuantifierRef:
    if isinstance(p, smt.QuantifierRef):
        vs, body = open_binder(p)
        if p.is_forall():
            bset = ProofSet(body)
            f = smt.FreshConst(smt.ArraySort(*[v.sort() for v in vs], blam.sort()))
            return smt.Lambda([f], smt.ForAll(vs, bset(f(*vs))))
        elif p.is_exists():
            bset = ProofSet(body)
            f = smt.FreshConst(smt.ArraySort(*[v.sort() for v in vs], blam.sort()))
            return smt.Lambda([f], smt.Exists(vs, bset(f(*vs))))
    elif smt.is_and(p):
        subproofsets = [ ProofSet(c) for c in p.children()]
        psorts = [p.sort() for p in subproofsets]
        smt.Tuple(psorts)
        smt.Lambda([p], smt.And(*[subproofset(p[n]) for subproofset in zip(subproofsets, range(len(subproofsets))) ]))
    elif smt.is_or(p): ...
        # which one
    elif smt.implies(p):
        hyp, conc = map(ProofSet, p.children())
        f = smt.FreshConst(smt.ArraySort(hyp.domain(), conc.domain()))
        x = smt.FreshConst(hyp.domain())
        return smt.Lambda([f], kd.QForAll([x], hyp(x), conc(f(x))))

# subtyping

algebraic datatypes
examples.
Sigma combinator.
Dependent records


Non recursive structs actually would be easy


https://isabelle.in.tum.de/dist/library/HOL/HOL-Algebra/Congruence.html#Congruence.partial_object|type partial_object / carrier
https://isabelle.in.tum.de/dist/library/HOL/HOL-Library/FuncSet.html#FuncSet.funcset|const Pi
https://isabelle.in.tum.de/dist/library/HOL/HOL/Finite_Set.html#Finite_Set.finite|const finite

eq_class https://isabelle.in.tum.de/dist/library/HOL/HOL-Algebra/Congruence.html#Congruence.eq_classes|const

https://pmc.ncbi.nlm.nih.gov/articles/PMC7324036/ Reasoning About Algebraic Structures with Implicit Carriers in Isabelle/HOL

https://en.wikipedia.org/wiki/Concrete_category 

In [ ]:
from typing import Protocol
class PartialObject(Protocol):
    carrier : smt.ArraySortRef

class EqObject(PartialObject, Protocol):
    eq : smt.FuncDeclRef

    #? @classmethod. I don't need implementation to implement this
    def elem(self, x, A):
        y = smt.FreshConst(x.sort())
        return smt.Exists([y], A[y], eq(x,y))

    def set_eq(self, A, B):

class EqObjectSection():
    def elem(P : EqObject, x, A):
    
def elem(P : EqObject, x, A):
    """
    Return a proof that x is an element of A.
    """
    y = smt.FreshConst(x.sort())
    return smt.Exists([y], A[y], P.eq(x,y))


    

In [ ]:
#def Lattice(A : SubSort):


def FreshFromSet(A, prefix="c"):
    assert A.range() == smt.BoolSort(), "A must be a set sort"
    return smt.FreshConst(A.domain(), prefix=prefix)

def SetFreshVar(name, A):
    assert A.range() == smt.BoolSort(), "A must be a set sort"
    return kd.FreshVar(name, A.domain())

def SetConst(name, A):
    assert A.range() == smt.BoolSort(), "A must be a set sort"
    return smt.Const(name, A.domain())

def partial_object_lemmas(P : PartialObject):
    SetFreshVar("f")
    P.carrier


class 



class Lattice(Protocol):
    carrier : smt.ArraySortRef # 
    meet : 
    join :


In [ ]:
def Refine(dt : smt.SortRef, tele : Telescope, res_sort):
    # register type as refined.
    # if algebraic datatype, register constructors and recognizers as refining.
    for i in range(dt.num_constructors()):
        prove_sig(dt.constructor(i)),  
        recognizer
    datatype.call_dict[dt] = smt.Lambda(tele.vars, smt.Lambda([x], res_sort[x])) 

In [ ]:
def TStruct(name, fields):
    #sorts = [ isinstance()]
    tele = normalize(fields)
    dt = kd.Struct(name)
    x = smt.FreshConst(dt.sort())
    dt.family = smt.Lambda([x], )

In [ ]:

def InductiveRel(name: str, *params: smt.SortRef) -> smt.Datatype:
    dt = Inductive(name)

    relname = name.lower()
    olddeclare = dt.declare
    preds = []  # tuck away extra predicate

    def declare(
        name, *args, pred=None
    ):  # TODO: would it ever make sense to not have a pred?
        olddeclare(name, *args)
        preds.append(pred)

    dt.declare = declare

    oldcreate = dt.create

    def create_relation(dt):
        """
        When inductive is done being defined, call this function
        """
        ev = smt.FreshConst(dt, prefix=name.lower())
        rel = smt.Function(relname, dt, *[x.sort() for x in params], smt.BoolSort())
        cases = []
        for i in range(dt.num_constructors()):
            precond = dt.recognizer(i)(ev)  # recognize case of the evidence
            pred = preds[i]  # In this case, this predicate should be true
            if pred is None:
                res = smt.BoolVal(True)
            elif isinstance(pred, smt.ExprRef):
                res = pred
            else:
                args = [dt.accessor(i, j)(ev) for j in range(dt.constructor(i).arity())]
                res = pred(*args)
            cases.append((precond, res))
        args = [ev]
        args.extend(params)
        rel = kd.define(relname, args, kd.cond(*cases))
        return rel

    def create():
        dt = oldcreate()
        dtrel = smt.Function(relname, dt, *[x.sort() for x in params], smt.BoolSort())
        rel.register(
            dt, lambda *args: dtrel(*args)
        )  # doing this here let's us tie the knot inside of lambdas and refer to the predicate.
        dtrel = create_relation(dt)
        dt.rel = dtrel
        return dt

    dt.create = create
    return dt

{is(Nil): |= ForAll(x!4,
        Implies(And(rvec(x!4, n!0)),
                Lambda(b,
                       Implies(b,
                               If(is(Nil, x!4),
                                  n!0 == 0,
                                  If(is(Cons, x!4),
                                     rvec(tl(x!4), n!0 - 1),
                                     unreachable!3))))[is(Nil,
                     x!4)])),
 is(Cons): |= ForAll(x!4,
        Implies(And(rvec(x!4, n!0)),
                Lambda(b,
                       Implies(b,
                               If(is(Nil, x!4),
                                  n!0 == 0,
                                  If(is(Cons, x!4),
                                     rvec(tl(x!4), n!0 - 1),
                                     unreachable!3))))[is(Cons,
                     x!4)]))}

In [1]:
from kdrag.contrib.telescope import *
from kdrag.all import *

def register_inductive(T : SubSort):
    dt = subsort_domain(T)
    x = smt.FreshConst(dt, prefix="x")
    for i in dt.num_constructors():
        recog = dt.recognizer(i)
        constr = dt.constructor(n)
        accs = [dt.accessort(i,j) for j in constr.arity()]
        prove_sig(recog, [(x, T[x])], T.rel(x, *[acc(x) for acc in accs])) # abstract in the left, unfolded in the right.
        # Then, If casing on the recognizer, we drive the well formedness condition forward one step.
        #for j, acc in enumerate(accs):
        #    prove_sig(acc, [(x,smt.And(T[x], dt.recognizer(i,j)(x))], case[j])
n = smt.Int("n")
x = smt.Real("x")
RVec = kd.InductiveRel("RVec", n)
RVec.declare("Nil", pred=n == 0)
RVec.declare("Cons", ("hd", smt.RealSort()), ("tl", RVec), pred=lambda hd, tl: tl.rel(n - 1))
RVec = RVec.create()
RVec.rel.defn

|= ForAll([rvec!0, n],
       rvec(rvec!0, n) ==
       If(is(Nil, rvec!0),
          n == 0,
          If(is(Cons, rvec!0),
             rvec(tl(rvec!0), n - 1),
             unreachable!1)))

       myappend(n!31, m!32, u!34, v!35) ==
       Concat(u!34, v!35)) to ForAll([n!37, m!38, u!40, v!41],
       myappend(n!37, m!38, u!40, v!41) ==
       Concat(u!40, v!41))


|= Implies(And(Length(l!39) == x!36,
            Length(v!41) == n!37,
            Length(u!40) == m!38),
        Lambda(l!39, Length(l!39) == n!37 + m!38 + x!36)[myappend(x!36,
                   n!37 + m!38,
                   l!39,
                   myappend(n!37, m!38, v!41, u!40))])

In [15]:
ulen = smt.Length(u)

app = define("myappend", [(u, Vec(ulen)), (v, Vec(smt.Length(v)))], Vec(ulen + smt.Length(v)), 
                    smt.Concat(u,v))
app.pre_post

|= ForAll([u!40, v!41],
       Implies(And(Length(u!40) == Length(u!40),
                   Length(v!41) == Length(v!41)),
               Lambda(l!39,
                      Length(l!39) ==
                      Length(u!40) + Length(v!41))[myappend(u!40,
                          v!41)]))

In [ ]:
from kdrag.contrib.telescope import *


n = kd.kernel.SchemaVar("n", smt.IntSort())#smt.Int("n")
m = smt.Int("m")
Nat = smt.Lambda([n], n >= 0)
Pos = smt.Lambda([n], n > 0)
inc = define("inc", [(n,Nat)], Pos, n + 1)
inc.pre_post

pred = define("pred", [(n, Pos)], Nat, n - 1)

myid = define("myid", [(n, Nat)], Nat, pred(inc(n)))
kd.prove(myid(n) == n, by=[inc.defn, pred.defn, myid.defn])
#kd.prove(myid(n) == n, by=[inc.pre_post, pred.pre_post, myid.pre_post])

Implies(And(n!57 >= 0), Lambda(n!57, n!57 > 0)[n!57 + 1])
Implies(And(n!57 > 0), Lambda(n!57, n!57 >= 0)[n!57 - 1])
Implies(And(n!57 >= 0),
        Lambda(n!57, n!57 >= 0)[pred(inc(n!57))])


LemmaError: (myid(n!57) == n!57, 'Countermodel', [n!57 = -1,
 myid = [-1 -> -2, else -> 0],
 pred = [else -> 0],
 inc = [else -> 2]])

In [ ]:
def TImplies(ctx: Telescope, conc: smt.BoolRef) -> smt.BoolRef:
    tele = normalize(ctx)
    return smt.Implies(smt.And([P for _, P in tele]), conc)

In [ ]:
def tdefine(name, tele, subset, body):
    

In [ ]:
def Id(x,y):
    return smt.Lambda([p], x == y)
n = smt.Int("n")
Nat = smt.Lambda([n], n >= 0)

In [ ]:
type Ctx = frozenset[tuple[smt.ExprRef, smt.BoolRef]]

In [ ]:
ann.pre_post = smt.ForAll([x], T[x], ann(x, T) == x)

# quote

In [ ]:
def quote_sort(s : smt.SortRef):
    T = smt.DeclareSort("'" + s.sexpr())
    return T
def quote(t : smt.ExprRef):
    assert not isinstance(t, smt.QuantifierRef) # we could handle this though.
    T = quote_sort(t.sort())
    decl = t.decl()
    children = [quote(c) for c in t.children()]
    Targs = [c.sort() for c in children]
    absdecl = smt.Function(decl.name(), *Targs, T)
    qt = absdecl(*children)
    qt.unquote = t

    interp = smt.Function("unquote", qt.sort(), t.sort())
    qt.ax = kd.axiom(interp(qt) == t)

    return qt

x + y * z
quote(x+y*z).unquote

x + y*z

In [ ]:
def reify(t : smt.ExprRef):
    # maybe it should be a python value coming in? And use kdrag.reflect.reify?
    assert smt.is_value(t)
    qt = quote(t)
    reify_decl = smt.Function("reify", t.sort(), tq.sort())
    return kd.axiom(reify_decl(t) == qt)

def unquote_reify(t):
    kd.axiom(reify(reflect(t)) == t)


In [ ]:
unquote_sort = {}

"""
def unquote_sort(T : smt.SortRef):
    name = T.name()
    assert name.startswith("'")
    name = name[1:]  # remove leading quote
    if name.startswith("'"):
        return smt.DeclareSort(name)
    elif name == "Real":
        return smt.RealSort()
    elif name == "Int":
        return smt.IntSort()
    elif name == "Bool":
        return smt.BoolSort()
    else:
        raise ValueError(f"Unimplemented sort name: {name}")    
"""
def interp_ax(s : smt.SortRef):
    T = quote_sort(s)
    interp = smt.Function("interp", T, s) 
    reify = smt.Function("reify", s, T)
    x = smt.Const("x", s)
    return kd.axiom(smt.ForAll([x], interp(reify(x)) == x))

def interp_ax(e):
    qe = quote(e)
    interp = smt.Function("unquote", e.sort(), unquote[e.sort()].sort())
    return kd.axiom(interp(qe) == e)
unquote = {}  # interp_table?
# could save away mapping from quoted to unquoted. But then can't interpret anything new made?
# But if I'm actually metaprogramming, just do it in python. This is for internalization.
def quote_sort(s : smt.SortRef):
    T = smt.DeclareSort("'" + s.sexpr())
    #unquote_sort[T] = s
    return T
def quote(t : smt.ExprRef):
    assert not isinstance(t, smt.QuantifierRef)
    T = quote_sort(t.sort())
    decl, children = t.decl(), t.children()
    Targs = [quote_sort(c.sort()) for c in children]
    absdecl = smt.Function(decl.name(), *Targs, T)
    t1 = absdecl(*[quote(c) for c in children])
    interp = smt.Function("unquote", t1.sort(), t.sort())
    t1.ax = kd.axiom(interp(t1) == t)
    t1.unquote = t
    #unquote[t1] = t
    return t1

x + y * z
quote(x+y*z).sort()

#def interp(t : smt.ExprRef):

is_true = smt.Function("is_sat", quote_sort(smt.BoolSort()), smt.BoolSort())

def is_true(t : smt.ExprRef, by=[]):
    # is_true is negation is unsat.
    #s = smt.Solver()
    #s.add(smt.Not(interp(t)))
    #if s.check() == smt.sat:
    #    return kd.axiom(is_true(t) == False, reasons=[])
    p = kd.prove(interp(t), by=by)
    return kd.axiom(is_true(t) == True, reasons=[p])
    

unquote[quote(x+y*z)]



x + y*z

# smt parse
Take all keywords, mangle them.
Use regular parser
turn 
sexp parse

hmm. doesn't actually work.

cmds = [
    smt.Function("declare-fun", name, args, )
]


define_fun


# sequent extract



In [ ]:
from dataclass import dataclass
@dataclass
class Sequent():
    hyps
    concs
def ProofTree():
    judgement : Sequent
    rule : object
    children : list['ProofTree']



def prove(e : smt.ExprRef):
    
    

# define_fun

Maybe doing this could help me understand those early chapters in intro hott

A primitve axiom style?


Like does induction over Nat really require dependent type in some sense?
Also acl2 like ?


def_thm(eqs)
    # define_fun (which deomnstrates termination)
    # but then show it is always True using apporpriate induction principle


https://www.cse.chalmers.se/~ulfn/papers/thesis.pdf
inaccessible patterns

Coquand 92 https://wonks.github.io/type-theory-reading-group/papers/proc92-coquand.pdf
Guguen 2006 https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/99.pdf  Eliminating Dependent Pattern Matching
Mcbride 1999 https://ncatlab.org/nlab/files/conor-mcbride-thesis.pdf
View from Left paper relevant?

Hmm. Dependent pattern matching. indices become unification problems
Case splitting tree. And finding the accessor patterns of the original guy. THis is like my implementation of Pattern match. But I suppose you could go into recursors. Hmm.
Telescopes as tries makes sense because I was distinguishing by value. Hmm.
Individual equations vs all equations

Sozeau


https://jesper.sikanda.be/files/pattern-matching-without-K.pdf pattern macthing wiuthout K. Jesper Cockx




In [ ]:


import kdrag.theories.list as lst
IntList = lst.List(smt.IntSort())
a,b,c = smt.Consts("a b c", IntList)
#rev = smt.Function("rev", IntList, IntList)
#define_fun([rev(IntList.Nil) == IntList.Nil,
#            rev(lst.Cons(a, lst.Cons(b, c))) == lst.Cons(rev(lst.Cons(b, c)), a)
#            ])
x = smt.Int("x")
append = smt.Function("append", IntList, IntList, IntList)
append = define_fun([append(IntList.Nil, b) == b,
            append(lst.Cons(x, c), b) == lst.Cons(x, append(c, b))
            ])
append.case0
append.case1

       add(c!319, c!320) ==
       If(And(is(Z, c!319)),
          c!320,
          If(And(is(S, c!319)),
             S(add(pred(c!319), c!320)),
             unreachable!321))) to ForAll([c!342, c!343],
       add(c!342, c!343) ==
       If(And(is(Z, c!342)),
          c!343,
          If(And(is(S, c!342)),
             S(add(pred(c!342), c!343)),
             unreachable!344)))
       append(c!330, c!331) ==
       If(And(is(Nil, c!330)),
          c!331,
          If(And(is(Cons, c!330)),
             Cons(head(c!330), append(tail(c!330), c!331)),
             unreachable!332))) to ForAll([c!353, c!354],
       append(c!353, c!354) ==
       If(And(is(Nil, c!353)),
          c!354,
          If(And(is(Cons, c!353)),
             Cons(head(c!353), append(tail(c!353), c!354)),
             unreachable!355)))


|- ForAll([c, x, b],
       append(Cons(x, c), b) == Cons(x, append(c, b)))

# partial / extensible


Open predicates gives me internalized prolog.
Could this be useful for the things typeclasses are used for? Automated search?
Extensible datatypes + open functions does help. The integer id is a witness in the 
https://ocaml.org/manual/5.3/extensiblevariants.html

Open Predicates
Open Functions
Open datatypes
https://www.cs.ox.ac.uk/people/ralf.hinze/talks/Open.pdf
https://www.cs.ox.ac.uk/ralf.hinze/publications/PPDP06.pdf
best fit matching... hmm.

Open patterns.

https://srfi.schemers.org/srfi-262/srfi-262.html#examples pattern matyching. Extensible pattern macthing / views. Hmm.
https://arxiv.org/abs/1106.2578  Extensible Pattern Matching in an Extensible Language. Tobin
Universes are open types.
Views was kind of an interesting angle on isomorphism. Snoc Cons views. Different reps of nats, etc. Quotienting.
"The original proposal for extensible pattern matching is Wadler’s ”views” [25], presented in the context of Haskell. In this system, a view is a pair of an injection and projection from an abstract type to an algebraic datatype. Views are intended to support data abstraction in conjunction with pattern matching."

https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/active-patterns active patterns in F# https://en.wikibooks.org/wiki/F_Sharp_Programming/Active_Patterns  https://stackoverflow.com/questions/56536157/what-is-the-main-point-of-active-patterns https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/p29-syme.pdf Extensible Pattern Matching Via a Lightweight Language Extension https://github.com/fsharp/fslang-suggestions/issues/968#issuecomment-764636595 https://github.com/dsyme/dsyme-presentations/blob/master/design-notes/notes-on-learning-fsharp-2.md
Kind of reminds me of TypeIs type guards.

To what degree can actual OO features be applied to a "logical" (pure?) system. There are coinductives. Subtyping. Inheritance. I think these are so confusing that boltiong them on the kernel is fraught with peril, but they can be compiled metafeatures.

https://ghc.gitlab.haskell.org/ghc/doc/users_guide/exts/view_patterns.html 

Maybe views could work as a mechanism.  T <=> {head : int, tail : T} is kind of a view 
section vs retract.  f(g(x)) ~ x. Which equality ?  I guess ~ could be a permutation or something?

I could maybe bolt views into user propagators.
OTT is about making views invisible (?) Deriving the needed casts/coercions.
call 
box(unbox(x))
force(thunk(x))
Lambda([n], T) won't work.


{head = n, tail = box(view(repeat(n))) }  = view(repeat(n))



unbox(box(x)) = x
box(unbox(x)) =? x.  No probably not. What if you boxed it some other way

```python

views = {}

def define_view(inj, proj, by=[]):
views.append(inj,proj, kd.prove(proj(inj(x)) == x)) #? left or right 

```

https://dl.acm.org/doi/abs/10.1145/3674627 Minikanren extensible patterns


Somehow extesnibility veers into object oriented.

https://dl.acm.org/doi/abs/10.1145/3649836 Persimmon: Nested Family Polymorphism with Extensible Variant Types . NEPLS poster. Interesting object orietned bibliography

https://dl.acm.org/doi/10.1145/1159803.1159836  Extensible programming with first-class cases. extensible records and extensible sums are dual

Familia: unifying interfaces, type classes, and family polymorphism  https://dl.acm.org/doi/10.1145/3133894

virtual types Kresten Krab Thorup. 1997. Genericity in Java with virtual types. https://link.springer.com/chapter/10.1007/BFb0053390

nested inheritance

Mads Torgerson. Virtual types are statically safe.

associated types  Chakravorty 2005

Object-Oriented Programming in the Beta Programming Language - Madsen https://en.wikipedia.org/wiki/BETA_(programming_language) https://beta.cs.au.dk/ When why and how of beta https://dl.acm.org/doi/10.1145/1238844.1238854

https://link.springer.com/chapter/10.1007/3-540-45337-7_17 ernst 2001  Family Polymorphism

Scalable extensibility via nested inheritance https://dl.acm.org/doi/10.1145/1028976.1028986

Cardelli badai https://www.amazon.com/Theory-Objects-Monographs-Computer-Science/dp/0387947752 1996 Theory of objects

https://mitpress.mit.edu/9780262526326/theoretical-aspects-of-object-oriented-programming/ Theoretical Aspects of Object-Oriented Programming
Types, Semantics, and Language Design - mitchell and Gunter

could seal to create an actual function definition. Later seals create a different but related definition.
Likewise could seal the cases... Hmm. So a global program compilation kind of thing. We can fullly defunctionalize if we do so.
The results of the different seals have to be ocnsistent, so we could have an axiom:
myfun_ver1(x) == y => myfun(inj(x)) == y





```python
class ExtensibleInductive():
    current : int = 0
    def __init__(self, name, *sorts):
        self.T = self.Record(name, ("index", smt.IntSort()), *sorts)
    def declare(self, name):


Extsenbile("Stream", ("head", IntSort()), ("tail", smt.Array( , ?))


class ExtensibleFun(): # OpenFunction
    decl :
    args : 
    cases = []
    def __init__(self, *args):
        decl = smt.Function(name, smt.IntSort(), *args) # add tag
    def extend(self, cond, body)
    def extend(self, body):
        kd.define(name + "case" + len(cases), args, smt.If(ind == len(cases), body, nextcase(*args))

    # I could narrow the arguments, but not change them?


# maybe code would want more data? You want to scope where the codes came from
class Code():
    current : int
    decode = pdefine("decode")



```





Override vs underride semantics vs enforce disjointness.
It is weird and probably bad 

I have stale axiom around.

Is the undefined version becoming more defined too?

Hmm. COnsidered as a systematic way of making new undef function. Interesting.
foo = if(c, e1, foo_undef1)

It's kind of just a way to invert definition ordering then. Is this even useful?

foo_undef1 = yada.

Oh yeah and the unfolding mechanism still works. Huh.

But for condictuvies or sets it will almost never be able to push through the conditionals. When are sets _not_ equal.


open predicates like prolog. We can only prove True because of outer Or
Or(
    case1, case2, undef_case
)

How to make minikanren extensible like prolog. Need open functions.
Re opening the clark completion form.

Yes, how do you ever know something isn't in predicate? Seal off. Observations?
undef == False would seal off forever. Hmm.
partial seal off?

```python
class OpenPred():
    pred : smt.FuncDeclRef
    args : list[smt.ExprRef]
    cases : list[smt.BoolRef]
    _undef : smt.FuncDeclRef
    #ax : kd.Proof

    def extend(self, body):
        new_undef = smt.FreshFunction()
        kd.define(self.undef.name(), args, smt.Or(body, newundef(*args)))
        self.undef = new_undef

    def __call__(self, *args):
        return self.pred(*args)
```

Hmm. Is this superior to closed function form using If?
Or(Implies(cond, f(args)  == body1)
   Implies(cond2, f(args) == body2)
   Implies(cond3, f(args) == )
)
Or form can start allowing f to have multiple models if clauses overlap. Will we be able to ever prove anything? No I guess.

vs

And(Implies(cond, f(args)  == body1)
   Implies(cond2, f(args) == body2)
   Implies(cond3, f(args) == )
)
And form can have insistency if cond overlap
Makes unfolding hard / impossible.


elem(x,A) ==

pdefine("elem",[x,A], A == pair(y,z), Or(x == y, x == z))

An open set of equations seems reasonable
e

pdefine("head", [s],  s == repeat(n), n)
pdefine("tail", [s],  s == repeat(n), repeat(n))


but it's hard to prove that streams are or aren't equal to other streams.

head(repeate(n), n) == True



In [ ]:
def pdefine(name, args, cond, body):
    sorts = [arg.sort() for arg in args] + [body.sort()]
    f = smt.Function(name, *sorts)
    undef = smt.FreshFunction(name + "_undef", *sorts)
    newbody = smt.If(cond, body, undef(*args))
    if f not in kd.kernel.defns:
        return kd.define(name, args, newbody)
    else:
        defn = kd.kernel.defns[f]
        #kd.prove(smt.Not(smt.And(cond, defn.defined))) # disjoint
        #newbody = smt.substitute(newbody, *zip(args, defn.args))
        old_undef = kd.define(defn.undef.name(), args, newbody)
        newbody = smt.substitute(defn.body, (defn.undef, newbody))
        defn._replace(body=smt.If(cond, body, defn.body))
        # we don't _have_ to replace ax.
        ax = kd.prove(smt.ForAll(args, f(*args) == newbody, by=[defn.ax, old_undef.defn]))
        f.defn = ax
        return f



# termination

Make a measure checking thing.
structural recursion




In [ ]:
#def lex_less(subargs, args):
#    all( subargs[i] <= args[i] for i in range(len(subargs) - 1) ) and subargs[-1] < args[-1]

def arg_dec(cases):
    N = len(cases[0][0])
    for i in range(N):
        if all(kd.utils.is_strict_subterm(subargs[i], args[i]) for args,subargs in cases):
            return i # ith argument is decreasing in all cases.
    return False
    #patterns = [[kd.util.is_subterm(x,y) for x,y zip(arg,subargs)] for args,subargs in cases]
    #for perm in itertools.permutations(n):
    #    if all(   )

#(False, True) < (True, True, True)




SyntaxError: invalid syntax. Perhaps you forgot a comma? (880595545.py, line 5)

In [ ]:
def lex_dec(vs, lhs, rhs):
    if lhs.sort() == smt.IntSort():
        kd.prove(smt.ForAll(vs, 0 <= lhs))
        kd.prove(smt.ForAll(vs, lhs >= rhs))
    else:
        raise ValueError("Unsupported sort for lexicographic decision procedure")
x,y = smt.Ints("x y")
lex_dec([x], )

# for a kd.define, we need to know every subcall is less than original.
# 


# copattern

Extensible definitions
Rewrite rules in simp?


I guess a copattern is a simpler thing than full rewriting


I guess non-deep matching is as simple

add2(S(n)) = 

patterns[add2][S] = ([n], S(S(n)))

https://wiki.c2.com/?ClosuresAndObjectsAreEquivalent
objects and messages
Message = Head | Tail
Response = Int(n) | Stream(s)
repeat(n)[Head] = 

define("repeat", n, Lambda([Msg], If(msg == Head, n, msg==Tail, repeat(n))))

A GADT could maybe make this work
  Head : Msg Int
  Tail : Msg Stream
type Stream = Msg 'a -> 'a

{
    a : T # Code type?
    head : T -> int
    tail : T -> T
}

head = lambda x, If(stream(n) == x, n, undefined)
But you can't extract n out of stream.





In [ ]:
coppatterns = {}
coppatterns[head][repeat] = ([n], n)
coppatterns[tail][repeat] = ([n], repeat(n))




def copattern(pat, rhs, t):
    assert pat.decl() == t.decl()
    zip(pat.children(), t.chidren())

In [ ]:
constructors = {}

# honestly it might be better to do things this way. Instead of searching through junk. I dunno.
List = smt.DeclareSort()
nil = smt.Const("nil", List)
cons = smt.Function("cons", smt.IntSort(), List, List)

constructors[List] = [nil, cons]

In [ ]:
observations = {}

Stream = smt.DeclareSort("Stream")
head = smt.Function("head", Stream, smt.IntSort())
tail = smt.Function("tail", Stream, Stream)
observations[Stream] = [head, tail]

zeros = smt.Const("zeros", Stream)

codefns = {}
def codefine(name, args, obs): # codefine is more like define_fun than it is like define.
    T = obs[0].arg(0).arg(0).sort()
    decl = smt.Function(name,  [arg.sort() for arg in args] ,T)
    assert T in observations
    axs = []
    for n, obeq in enumerate(obs):
        assert smt.is_eq(obeq)
        lhs,rhs = obeq.children()
        ob = lhs.decl()
        assert ob == observations[T][n]
        axs.append(smt.ForAll(args, ob))
        codefns[(decl, ob)] = (vs, rhs)
    return axs


zeros = codefine("zeros", [],
         [head(zeros) == 0,
          tail(zeros) == zeros])

wrap = smt.Function("wrap", StreamT, Stream)
codefine("zeros", wrap(StreamT(0, zeros)))


#codefine

Ok new idea.

Inductive observcations

Stream A ~ Nat -> A

head f = f Z
tail f = \o -> f (S o)

Stream2 A B = { ob1 : A, ob2 : B, next : stream2} -->  {ob1 : Nat -> A, ob2 : Nat -> B}



ITree = {l : itree, r : itree, v : A}
type Pos = L Pos | R Pos | Here
type ITree = Pos -> A
val x = x 
l x = \p -> x (L p)
r x = \p -> x (R p)

-> x (snoc L p)

l (r x) = \p -> x (L (R p))
but this is probably inverted from what we want.
But x is probably implemented as a fold over obs so 
Sequences of observations.

double : s, inc: s, val : Nat
doub (inc x)  = Inc Doub

SeqSort(ObsEnum)

head (all n) = n
tail (all n) = all (n+1) 

all n = \o -> rec (+1) n o


l (doub n) = doub (2 * n)
r (doub n) = doub (n + 1)
v (doub n) = n

v (l (r (doub 0))) = 1

[L, R, Here]
Hmm could even use seq.Fold

doub n = seq.Fold n (\ acc o -> | L ->  )


generators can be turned into `nat -> A`.
thermometer continuations


If I have Seq[Obs] then I can intyernally use RegEx for some of them.

\o -> smt.If(RegEx(p)[0], 10, RegEx(p)(0))
This is kind ofa way of encoding automata with output. Regex only https://en.wikipedia.org/wiki/Moore_machine . or is it a mealy where regex is i?


https://inria.hal.science/hal-03671876
https://inria.hal.science/hal-03689027v2/document toiwards corecursion in coq

https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-304.pdf  Mechanizing Coinduction and Corecursion in Higher-order Logic


BB(5)
Renormalization of the cutoff of "small number". Elegant and inelegant methods. Stable and unstable properties?
Probalistic logic?
https://www.uvm.edu/~uvmsrc/archive/2015/abstracts/uvmsrc2015.pdf dummett wang's paradox
induction should have a small epsilon reducing the truth. Induction is a form of difference equation.
Renomralization of truth




# solver for

Hmmm. Wait. This is a guassian eliminator?




In [ ]:
import z3
def solve(eqs, x):
   s = z3.SimpleSolver()
   s.add(eqs)
   return s.solve_for([x])
x,y,z = z3.Reals('x y z')
solve([x + y == z], x)

[]

In [ ]:
import z3
x,y,z = z3.Reals('x y z')
s = z3.Solver()
s.add(x + y == z)
print(z3.get_version_string())
print(s.solve_for([x]))
#s.solve_for1(x)


4.15.0
[]


In [ ]:
z3.IntVal(3).py_value()

3

In [ ]:
z3.get_version_string()

'4.15.0'

In [ ]:
import z3
x,y,z,p = z3.Reals('x y z p')
s = z3.SimpleSolver()
s.add(x + y == z)
s.add(p + y + 5 == z)
s.check()
print(s.solve_for([x]))

[(z, p + y + 5, And(y + -1*z + p <= -5, y + -1*z + p >= -5)), (x, z + -1*y, And(x + y + -1*z >= 0, x + y + -1*z <= 0))]


https://github.com/Z3Prover/doc/tree/master/synthesiz3
https://github.com/Z3Prover/doc/commit/690d88ba43a7d706d174da2c48279a5d16810a92

# Proof
z3 proof objects were an ast.
They even had modus ponens?
Hmm.
Look at z3 and cvc5 proof objects

https://ceur-ws.org/Vol-3185/paper9527.pdf simple proof format for SMT
https://ceur-ws.org/Vol-3455/invited1.pdf Challenges in SMT Proof Production and Checking for
Arithmetic Reasoning

In [ ]:
from kdrag.all import *
s = smt.Solver()
s.set("clause_proof", True)
f = smt.Function("f", smt.IntSort(), smt.BoolSort())
s.add(smt.ForAll([x], f(x)))
s.add(smt.Not(f(1)))
print(s.check())
p = s.proof()
p.decl()
assert not p.sort() == smt.DeclareSort("Proof")
p.sexpr()

unsat


'(let ((a!1 (forall ((x Int)) (! (f x) :pattern ((f x))))))\n(let ((a!2 (assumption (inst a!1 (not (f 1)) (bind 1) (gen 0))\n                       (or (not a!1) (f 1)))))\n  (proof-trail (assumption assumption a!1)\n               (assumption assumption (not (f 1)))\n               a!2\n               false)))'

In [ ]:
Proof = smt.DeclareSort("Proof")
andI = smt.Function("andI", Proof, Proof, Proof) # to build multicontext
prove = smt.Function("prove", smt.BoolSort(), Proof, Proof)
axiom = smt.Function("axiom", smt.BoolSort(), Proof)

#instan = smt.Function("instan", Proof, Proof)

# both need to be multiarity / parametric
#T = smt.TypeVar("T")
#herb = smt.Function("herb", smt.ArraySort(), Proof)
#instan = smt.Function("instan", Proof, Proof)





# cvxpy

But really, doesn't z3 offer this? It's about scale.
I guess it doesn't offer SDP
We might also want to verify cvxpy transforms like that lean project?

duality? 

In [ ]:
import cvxpy as cp
from kdrag.all import *
import operator

x,y = smt.Reals("x y")
binop = {
    (x <= y).decl() : operator.le,
    (x >= y).decl() : operator.ge,
    (x == y).decl() : operator.eq,
}



def tocvxpy(e : smt.ExprRef):
    if isinstance(e, smt.BoolRef):
        if e.decl() in binop:
            return binop[e.decl()](tocvxpy(e.arg(0)), tocvxpy(e.arg(1)))
        else:
            return 
        return cvxpy.Constant(e.py_value())

x = cp.Variable(1, name="x")
x


Variable((1,), x)

https://en.wikipedia.org/wiki/Sum-of-squares_optimization

In [ ]:
import z3
x,y = smt.Reals("x y")
z3.prove(x**2 - 4*x*y + 7*y**2 >= 0)


proved


# sympy
We're stuck. Just let soundness go

Also tying together a solver could be fun

taylor series





In [ ]:
import kdrag as kd
import kdrag.theories.real as real
from kdrag.theories.real.sympy import sympify, simplify, solve, kdify
from kdrag.all import *


x,y,z = smt.Reals("x y z")
sympify(x + y == z)
sympify(x + y <= z)
#implify(real.cos(x)**2 + real.sin(x)**2 == z)
simplify(real.cos(x) == 3)

solve([real.cos(x) == 3], [x])
#solve([real.exp(x) == x], [x])

t = sympify(real.deriv(smt.Lambda([x], real.cos(real.sin(x)))))

simplify(real.integrate(smt.Lambda([x], real.sin(x)), 0,x))


sympify(real.summation(smt.Lambda([x], real.exp(-x)), 0, y)) # hmm
simplify(real.deriv(smt.Lambda([x], real.cos(real.sin(x)))))
import sympy
a = sympy.Symbol('a')
kdify(sympy.Lambda(a,a))
simplify(smt.Lambda([x], 2 + x + 1))

class Lemma(kd.Lemma):
    def simp(self, at=None, unfold=False):
        super().simp(at=at, unfold=unfold)
        goalctx = self.top_goal()
        goal = goalctx.goal
        if at is None:
            newgoal = simplify(goal)
            print(newgoal)
            self.lemmas.append(kd.axiom(goal == newgoal, by=["sympy simplify"]))
            self.goals[-1] = goalctx._replace(goal=newgoal)
        return self.top_goal()
    def solve(self, vs):
        goalctx = self.goals.top_goal()
        return solve(goalctx.ctx, vs)
    #def solve_for(self, vs):
    #    contraints = self.goals[-1].ctx
    #    s = smt.SimpleSolver()
    #    s.add(contraints)
    #    return s.solve_for(vs)

l = Lemma(real.cos(x)**2 + real.sin(x)**2 == 1)
l.simp()
l.qed()




True


|- cos(x)**2 + sin(x)**2 == 1

In [ ]:
(smt.Lambda([x], x))

SympifyError: SympifyError: "cannot sympify object of type <class 'kdrag.reflect.KnuckleClosure'>"

In [ ]:
(-x).decl().kind()

520

In [ ]:
dir(smt.Lambda([x], x))
smt.Lambda(x,x)

Lambda(x, x)

In [ ]:
import sympy
a = sympy.Symbol('a')
a.name
def Lambda(x,y): return x,y
#sympy.lambdify([], sympy.Lambda(a, a), modules={"Lambda" : Lambda})()
sympy.E
isinstance(sympy.Lambda(a,2+a), sympy.Lambda)
sympy.Lambda(a,2+a).variables

(a,)

In [ ]:
sympy.Lambda(a, a + 7 * a).atoms()

{8, a}

In [ ]:
a

NameError: name 'a' is not defined

# eexists / uvars

uvars realkly belong to the total goal state not Goals.
tracking what is allowed in them uses the goal vars at the time.
Yikeys.
Can just collect junk to be included in prove(smt.Exists(uvars, ))

prove(smt.Exist)

kd.QExists(uvars, constraints, whatev)

Kind of need uvars to use solve_for and sympy stuff?


In [ ]:
class Lemma():
    uvars : set[smt.ExprRef]
    constraints : list[smt.BoolRef] # or go into lemmas?
    



def eexists()

# spacer

Can we use spacer for something? Inductive theorem proving?
https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/Fixedpoints
https://github.com/Z3Prover/z3/discussions/5013
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/Dagstuhl2019.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/FSU_2023.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py


DIY IC3 solver?
Needs interpolation right?
Could just use spacer for interp

https://github.com/Z3Prover/z3/blob/master/examples/python/mini_ic3.py
https://github.com/pddenhar/Z3-IC3-PDR

https://github.com/arbrad/IC3ref 

https://github.com/JakubSarnik/geyser
https://github.com/gipsyh/rIC3

aiger and btor2



In [ ]:
from kdrag.all import *
def solver_horn(chc):
    # https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py
    s = z3.SolverFor('HORN')
    s.set('engine', 'spacer')
    s.set('spacer.order_children', 2)
    s.add(chc)
    res = s.check()
    if res == z3.sat:
        return res, s.model()
    elif res == z3.unsat:
        return res, s.proof()
    else:
        return res, None
    
def interpolate(As, A, Bs, B, shared):
    # Uninterpreted predicate Itp over shared symbols
    Itp = z3.Function('Itp', [s.sort() for s in shared] + [z3.BoolSort()])

    # first CHC: A ==> Itp
    left = z3.ForAll([a for a in As], z3.Implies(A, Itp(shared)))
    # second CHC: Itp ==> !B
    right = z3.ForAll([b for b in Bs], z3.Implies(Itp(shared), z3.Not(B)))

    # run CHC solver
    res, answer = solve_horn([left, right])

    if res == z3.sat:
        return answer.eval(Itp(shared))
    else:
        return None





https://microsoft.github.io/z3guide/docs/fixedpoints/engineforpdr

In [ ]:
mc = smt.Function("mc", smt.IntSort(), smt.IntSort(), smt.BoolSort())
n,m,p = smt.Ints("n m p")
q1 = smt.Function("q1", smt.IntSort(), smt.IntSort(), smt.BoolSort())
clauses = [
    kd.QForAll([m], m > 100, mc(m, m - 10)),
    kd.QForAll([m,p,n], m <= 100, mc(m + 11, p), mc(p,n), mc(m,n)),
    #kd.QForAll([m,n], mc(m,n), n < 91, q1(m,n)),
]
solver_horn(clauses)



(sat, [mc = [else -> True], q1 = [else -> True]])

https://z3prover.github.io/papers/programmingz3.html#sec-horn-clause-solver

In [ ]:
s = smt.SolverFor("HORN")
Z = smt.IntSort()
B = smt.BoolSort()
mc = smt.Function('mc', Z, Z, B)
x, y, z = smt.Ints('x y z')
s.add(smt.ForAll(x, smt.Implies(x > 100, mc(x, x - 10))))
s.add(smt.ForAll([x, y, z], 
             smt.Implies(smt.And(x <= 100, mc(x + 11, y), mc(y, z)), 
                     mc(x, z))))
s.add(smt.ForAll([x, y], smt.Implies(smt.And(x <= 101, mc(x, y)), y == 91)))
s.add(smt.ForAll([x, y], smt.Implies(smt.And(x >= 101, mc(x, y)), x == y + 10)))
print(s.check())
print(s.model().eval(mc(x, y)))

sat
And(Not(y + -1*x <= -11),
    Not(y <= 90),
    Or(Not(y >= 92), Not(y + -1*x >= -9)))


## popper
https://github.com/logic-and-learning-lab/Popper/

Houdini

Is invariant (?)

def inv(P)
    for P in candidates()
        try:
            kd.prove(induct(P).arg(0))





# tableau / meson

For modal logics might be nice
Also smt boosted meson is interesting in and of itself.



# brute check
For anything 32bit or less we ought to just brute check it.

float16 for example. Would give me more confidence.
single variable float32 ops



# Solvers
##  isabelle
Graham was saying why not just download isabelle... Hmm.



```bash
emconfigure ./configure --enable-ho
emmake make AR="/home/philip/Downloads/emsdk/upstream/emscripten/emar rcs"
```

 And had to comment out in Makefile.vars the forced CC and AR values


https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho

bitwuzla
cvc5 
ytices
pysmt

bto2
moxi

implement model checking algorithms?



In [ ]:
import pyboolector as btor

btor.Parse("example.btor")



In [ ]:
import urllib.request
import os
import stat
filename = "eprover-ho"
#urllib.request.urlretrieve("https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho", filename)
st = os.stat(filename)
    # Add execute permissions for user, group, and others
os.chmod(filename, st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

In [ ]:
from kdrag.solvers import EProverTHFSolver
s = EProverTHFSolver()
s.add(x == y)
s.check()
s.res.stdout

b'# Preprocessing class: HSSSSMSSSSSNFFN.\n# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)\n# Starting new_ho_10 with 1500s (5) cores\n# Starting ho_unfolding_6 with 300s (1) cores\n# Starting sh4l with 300s (1) cores\n# Starting ehoh_best_nonlift_rwall with 300s (1) cores\n# new_ho_10 with pid 996928 completed with status 9\n# ho_unfolding_6 with pid 996929 completed with status 9\n# sh4l with pid 996930 completed with status 9\n# ehoh_best_nonlift_rwall with pid 996931 completed with status 9\n# Schedule exhausted\n# SZS status GaveUp\n'

In [ ]:
import subprocess
subprocess.run(["./eprover-ho", "--help"], check=True)


E 3.2.5-ho "Puttabong Moondrop"

Usage: eprover [options] [files]

Read a set of first-order (or, in the -ho-version, higher-order)
clauses and formulae and try to prove the conjecture (if given)
or show the set unsatisfiable.

Options:

   -h
  --help
    Print a short description of program usage and options.

   -V
  --version
    Print the version number of the prover. Please include this with all bug
    reports (if any).

   -v
  --verbose[=<arg>]
    Verbose comments on the progress of the program. This differs from the
    output level (below) in that technical information is printed to stderr,
    while the output level determines which logical manipulations of the
    clauses are printed to stdout. The short form or the long form without
    the optional argument is equivalent to --verbose=1.

   -o <arg>
  --output-file=<arg>
    Redirect output into the named file.

   -s
  --silent
    Equivalent to --output-level=0.

   -l <arg>
  --output-level=<arg>
    Select an outpu

CompletedProcess(args=['./eprover-ho', '--help'], returncode=0)

# egraph

Accept prolog and smtlib syntax?
like the good old days



## Mixins vs subclassing

GaussMixin()
t = GaussTheory()
EGraph(GaussTheory(), SympyTheory(), )

self.theories = []

def find(self, x):
    for t in theories:
        t.find(x)

def union():
    todo = []
    for t in theories:
        todo.extend(t.union())


## Lambda
alpha_norm before using add term.

Recursively carry params down through?
Eta long form.

A normal form?

Do slotted. Add some knd of permutation egraph?


eta_maximal form

In [ ]:
def eta_long(ctx, t : smt.ExprRef):
    if isinstance(t, smt.QuantifierRef):
        

## coegraph

Maybe use cvc5 codatypes
Datatypes already achieve the cocongruence propagation



## aptroot
https://mastodon.gamedev.place/@harold/114586279569895001




In [ ]:
from kdrag.all import *
Fruit = kd.Enum("Fruit", ["A", "B"])
import kdrag.theories.seq as seq_

FString = seq_.SeqSort(Fruit)

seq_.SeqVal([Fruit.A]*3) == seq_.SeqVal([Fruit.A, Fruit.B, Fruit.A])





## ILP extract
cvxpy translation?


import extraction gym?



## Gauss

Sympy, solve_for, passaegmath msolve



In [ ]:
class GaussEGraph():
    def __init__(self):
        self.eqs = smt.SimpleSolver() # Or sympy grobner basis.
    def union(self, x, y):
        if isinstance(smt.ArithRef):
            self.eqs

class GrobnerEGraph(EGraph):


In [ ]:

def sympy_simplify(self):
    for t in self.terms:
        if t.sort() == smt.RealSort():



## Extract

In [ ]:

from kdrag.all import *
from kdrag.solvers.egraph import EGraph
from collections import defaultdict
def extract(self, t0 : smt.ExprRef, cost_fun = (lambda _: 1)):
    inf = float("inf")
    best_cost = defaultdict(lambda: inf)
    best = {}
    while True:
        done = True
        # Terms are taking the place of enodes.
        for t in self.terms.values():
            eid = self.find(t)
            cost = cost_fun(t) + sum([best_cost[self.find(c)] for c in t.children()]) # cost_fun(t.decl()) ?
            if cost < best_cost[eid]:
                best_cost[eid] = cost
                best[eid] = t
                done = False
        if done:
            break
    #@functools.cache
    def build_best(t):
        t1 = best[self.find(t)]
        return t1.decl()(*[build_best(c) for c in t1.children()])
    return build_best(t0)

E = EGraph()
x,y,z = smt.Ints('x y z')
E.add_term(x + y)
E.rebuild()
assert extract(E, x + y).eq(x + y)
E.union(x + y, y)
E.rebuild()
assert extract(E, x + y).eq(y)




False

## Proof

In [ ]:
s = smt.Solver()
s.set(unsat_core=True)
a,b,c,d, e = smt.Ints('a b c d e')
id_map = {}
def tassert(a):
    tid = a.get_id()
    id_map[tid] = a
    #s.assert_and_track(a, smt.Bool(str(a)))
    s.assert_and_track(a, smt.Bool(str(tid)))
def core():
    core = s.unsat_core()
    return [id_map[int(str(c))] for c in core]


tassert(a == b)
tassert(b == c)
tassert(e == d)
#s.add(a == b)
#s.add(b == c)
#s.assert_and_track(a == b)
#s.assert_and_track(b == c, b == c)
s.add(a != c)
s.check()
#s.unsat_core()
type(core()[0])

z3.z3.BoolRef

In [ ]:
class EgraphProof(EGraph):
    def __init__(self):
        super().__init__()
        self.reasons = {}
        self.solver.set("unsat_core", True)
    def union(self, e1, e2, reason=None):
        super().union(e1, e2)
        if reason is not None:
            p = smt.FreshConst(smt.BoolSort())
            self.reasons[p.get_id()] = (e1,e2,reason)
            self.solver.assert_and_track(e1 == e2, p) 
    def add(self, expr, reason=None): # Use this indriectio nin Egrpha, so we can use assert and track
        tid = expr.get_id()
        self.terms[tid] = expr
        p = smt.FreshConst(smt.BoolSort())
        self.reasons[p.get_id()] = reason
        self.solver.assert_and_track(expr, p)
    def core(self):
        core = self.solver.unsat_core()
        return [self.terms[int(str(c))] for c in core]


Return instantiations of rules that make this possible
(vs, rule, lhs = rhs)


return ground rewrite system that'll do it? But then each rule needs to be itself dignified. bleh.

I could also use the trick from the third post of using z3 ematcher and grab the instans

Also it'd be neat to extract those into a reified proof object.


## Context
Egraph does support hypothetical equalities?
But I won't ematch unless I'm in that hypothetical situation

ctxs = []
ctx rebuilding - which ctx subsume, which 
Implies(ctx1, ctx2)
Implies(ctx2, ctx1)  ---> 

contextual rewrite rules.

Brute force contextual egraph

Ok so i can build it.
But then what is it for?
Well, if then else is a good example.

add_term(self, ctx, t):
    if smt.is_ite(t):
        ctx.append(t.arg(0)), t.arg(1)
        ctx.append(smt.Not(t.arg(1))), t.arg(0)



There's a lot of pruning possible in these queries.

In [ ]:
from kdrag.all import *

s = smt.Solver()
with s:
    s.add(False)
    print(s.check())
print(s.check())



unsat
sat


In [ ]:
    def rebuild(self):
        super().rebuild()
        ctxs = self.roots[smt.BoolSort()] & self.ctxs
        for ctx1 in ctxs:
            for ctx2 in ctxs:
                if ctx1 != ctx2 and self.subsumes(ctx1, ctx2):
                    self.solver.add(smt.Not(smt.Implies(ctx1, ctx2)))

In [ ]:
from kdrag.all import *
from kdrag.solvers.egraph import EGraph
def ctx_order(ctxs):
    s = smt.Solver()
    eq = []
    le = []
    for ctx1 in ctxs:
        for ctx2 in ctxs:
            s.push()
            s.add(smt.Not(ctx1 == ctx2))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                eq.append((ctx1,ctx2))
            s.push()
            s.add(smt.Not(smt.Implies(ctx1, ctx2)))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                le.append((ctx2,ctx1))

def subsumes(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(smt.Implies(ctx1, ctx2)))
    return s.check() == smt.unsat
def iff(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(ctx1 == ctx2))
    return s.check() == smt.unsat            


x,y,z = smt.Ints('x y z')

ctx1 = smt.And(x == y, y == z)
ctx2 = x == z
assert subsumes(ctx1, ctx2)
assert not subsumes(ctx2, ctx1)
iff(ctx1, ctx1)

class CtxEGraph(EGraph):
    ctxs : set[int]
    subsume : dict[int, set[int]] # "path" transitive closure

    # there's two versions. One where you have pattern vars in ctx or not. or if contexrt is pattern.
    # suearch over all subsumed contexts.
    # def ematch_ctx(self, vs, pctx, lhs):
    def ematch_ctx(self, vs, lhs):
        res = []
        for ctx in self.ctxs:
            #self.substitute(ctx, vs)
            with self.solver: # takes the push?
                self.solver.add(self.terms[ctx])
                res.extend((ctx, match_) for match_ in self.ematch(vs, lhs))
        return res
    #def ctx_sub():
    def rebuild(self):
        super().rebuild()
        oldctxs = self.ctxs
        for ctx1 in self.ctxs:
            for ctx2 in self.ctxs:
                if ctx2 in self.subsume[ctx1]:
                    continue
                else:
                    self.solver.push()
                    self.solver.add(smt.Not(smt.Implies(ctx1, ctx2)))
                    self.solver.pop()





In [ ]:
def add_term_quant(term):
    t = alpha_norm(term)
    todo = [([], t)]
    while todo:
        ctx, t = todo.pop()
        



# datalog
z3 has a datalog, but whatev. Take snakelog approach https://github.com/philzook58/snakelog

Also homommorphism finder. A la CSP / sqlite blog post

Pretty print into ASP?

spacer?


sqlite dump of binary
sqlite dump of clang ast or treesitter




In [ ]:
from kdrag.all import *
x,y,z = smt.Ints("x y z")
edge = smt.Function("edge", smt.IntSort(), smt.IntSort(), smt.BoolSort())
path = smt.Function("path", smt.IntSort(), smt.IntSort(), smt.BoolSort())
edgepath = rw.rule_of_expr(kd.QForAll([x,y], edge(x,y), path(x,y)))
trans = rw.rule_of_expr(kd.QForAll([x,y,z], edge(x,y), path(y,z), path(x,z)))


def compile_body(vs : list[smt.ExprRef], body : smt.BoolRef) -> str:
    todo = [body]
    env = {}
    froms = []
    wheres = []
    counter = 0
    while todo:
        rel = todo.pop()
        if smt.is_eq(rel):
            raise ValueError("Equality not supported")
        elif smt.is_not(rel):
            raise ValueError("Negation not supported")
        elif smt.is_or(rel):
            raise ValueError("Disjunction not supported")
        elif smt.is_and(rel):
            todo.extend(rel.children())
        elif smt.is_true(rel):
            continue
        elif smt.is_app(rel):
            name = rel.decl().name()
            args = rel.children()
            row_name = name + str(counter)
            counter += 1 
            froms.append(f"{name} AS {row_name}")
            for n, arg in enumerate(args):
                if arg in vs:
                    if arg in env:
                        wheres.append(f"{env[arg]} = {row_name}.x{n}")
                    else:
                        env[arg] = f"{row_name}.x{n}"
                else:
                    wheres.append(f"{row_name}.x{n} = {str(arg)}")
        else:
            raise ValueError(f"Unsupported expression: {rel}")
    return env, froms, wheres

def compile_rule(rule : kd.rewrite.Rule) -> str:
    env, froms, wheres = compile_body(rule.vs, rule.hyp)
    name = rule.conc.decl().name()
    selects = []
    for n,arg in  enumerate(rule.conc.children()):
        if arg in rule.vs:
            if arg in env:
                selects.append(f"{env[arg]} AS x{n}") # maybe select as keyword
            else:
                raise ValueError(f"Variable {arg} not found in body")
        else:
            selects.append(f"{arg} AS x{n}")
    froms = ", ".join(froms)
    wheres = " AND ".join(wheres)
    selects = ", ".join(selects)
    return f"""
    INSERT OR IGNORE INTO {name} SELECT DISTINCT {selects}
    {"FROM " + froms if froms else ""}
    {"WHERE " + wheres if wheres else ""}
    """

print(compile_rule(edgepath))
compile_rule(kd.rewrite.rule_of_expr(edge(1,2)))



    INSERT OR IGNORE INTO path SELECT DISTINCT edge0.x0 AS x0, edge0.x1 AS x1
    FROM edge AS edge0
    
    


'\n    INSERT OR IGNORE INTO edge SELECT DISTINCT 1 AS x0, 2 AS x1\n    \n    \n    '

In [ ]:
import sqlite3
class Datalog():
    def __init__(self):
        self.db = sqlite3.connect(":memory:")
    
    def declare_sig(self, sig : list[smt.FuncDeclRef]):
        for f in sig:
            primkey = "(" + ", ".join([f"x{i}" for i in range(f.arity())]) + ")"
            self.db.execute(f"""CREATE TABLE IF NOT EXISTS {f.name()} 
                            ({', '.join([f'x{i} {f.range().name()}' for i in range(f.arity())])}, 
                             PRIMARY KEY {primkey})""")

    def run(self, rule : kd.rewrite.Rule | smt.BoolRef):
        if isinstance(rule, smt.BoolRef):
            rule = kd.rewrite.rule_of_expr(rule)
        sql = compile_rule(rule)
        self.db.execute(sql)

s = Datalog()
s.declare_db([edge, path])
s.run(edge(1,2))
s.run(edge(2,3))
s.run(edgepath)
s.run(trans)
s.run(trans)
s.db.execute("SELECT * FROM path").fetchall()
        

[(1, 2), (2, 3), (1, 3)]

# compile out of smt2

Look at basic compiler books.
Tiger
Essentials
nanopass

smtcc :)
prescheme

undefined behavior.


We can make a meta layer list
https://pypy.org/posts/2024/07/toy-abstract-interpretation.html
https://pypy.org/posts/2022/07/toy-optimizer.html

In [ ]:

def ret(x):
    return "mov rdi, " + str(x) + "\n" + \


In [ ]:
%%file /tmp/test.c
#include <stdint.h>
uint64_t mytest(){
    return 42;
}


Writing /tmp/test.c


In [ ]:
! gcc -S -o /tmp/test.s /tmp/test.c && cat /tmp/test.s

	.file	"test.c"
	.text
	.globl	mytest
	.type	mytest, @function
mytest:
.LFB0:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movl	$42, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	mytest, .-mytest
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [ ]:
%%file /tmp/test.S

.globl mytest
mytest:
    mov $42, %rax
    ret


Overwriting /tmp/test.S


In [ ]:
!gcc -c -fPIC -o /tmp/test.o /tmp/test.S && ld -shared -o /tmp/test2.so /tmp/test.o && objdump -d /tmp/test.so


/tmp/test.so:     file format elf64-x86-64


Disassembly of section .text:

0000000000001000 <mytest>:
    1000:	48 c7 c0 2a 00 00 00 	mov    $0x2a,%rax
    1007:	c3                   	ret


In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("""uint64_t mytest();""")
lib = ffi.dlopen("/tmp/test2.so")
dir(lib)
print(lib.mytest())
del lib

def 



42


In [ ]:
World = smt.DeclareSort("World")
BV64 = smt.BitVecSort(64)
mem = smt.Function("mem", World, smt.ArraySort(BV64, BV64))

ret = smt.Function("ret", BV64, World, World)


type Block = list[smt.ExprRef]

class Module():
    

#class Block():
#   stmts : list[smt.ExprRef]



In [ ]:
x,y,z = smt.BitVecs("x y z", 64)
getarg = smt.Function("getarg", smt.IntSort(), smt.BitVecSort(64))
copy = smt.Function("copy", smt.BitVecSort(64), smt.BitVecSort(64)) # kd.define("copy", [x], x)
prog = [
    a := x + y,
    b := a + z,
    c := a - b,
]
def address_code(prog):
    seen = set()
    for stmt in prog:
        if all(c in seen for c in stmt.children()):
            seen.add(stmt)
        else:
            return False
    return True
# path assumptions could be interesting.

def esimplify(prog):
    uf = {}
    new_prog = []
    for n,stmt in prog:
        s = smt.Solver()


def register(prog):
    live = set()
    regalloc = {}
    availreg = ["r8", "rsi","rdi","rax"]
    for stmt in reversed(prog):
        availreg.append(regalloc[stmt])
        live.remove(stmt)
        for c in stmt.children():
            if c not in live:
                live.add(c)
                if availreg:
                    regalloc[c] = availreg.pop()
                else:
                    # spill
                    raise Exception("No registers available")

    




[x + y, x + y + z, x + y - (x + y + z)]

In [ ]:
from kdrag.all import *
one = smt.BitVecVal(1, 64)
zero = smt.BitVecVal(0, 64)

one + zero

def to_asm(e : smt.ExprRef):
    if isinstance(e, smt.BitVecNumRef):
        return "mov rdi, %d" % e.as_long()
    elif smt.is_add(e):
        to_asm(e.args[0])
        return [
            f"pop %rdi",
            r"add rdi, %d" % e.args[1].as_long()
        ]
    






1 + 0

# vectorize

pshufb
pext pdep

Aptroot's bot.

These chess guys are nuts
https://www.chessprogramming.org/BMI2#PEXTBitboards 


In [ ]:
# https://jamesbornholt.com/papers/diospyros-asplos21.pdf
BV16 = smt.BitVecSort(16)
BV8 = smt.BitVecSort(8)

x,y,z = smt.BitVecs("x y z", 16)
vadd = smt.Function('vadd', BV16, BV16, BV16)
vsub = smt.Function('vsub', BV16, BV16, BV16)
vmul = smt.Function('vmul', BV16, BV16, BV16)
vneg = smt.Function('vneg', BV16, BV16)
vmac = smt.Function('vmac', BV16, BV16, BV16, BV16)

vadd(x,y) == smt.Concat(smt.Extract(15, 8, x) + smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) + smt.Extract(7, 0, y))

vsub(x,y) == smt.Concat(smt.Extract(15, 8, x) - smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) - smt.Extract(7, 0, y))
vmul(x,y) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y))
vneg(x) == smt.Concat(-smt.Extract(15, 8, x), -smt.Extract(7, 0, x))
vmac(x,y,z) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y) + smt.Extract(15, 8, z),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y) + smt.Extract(7, 0, z))

smt.prove(smt.Concat(smt.Extract(15, 8, x), smt.Extract(7, 0, x)) == x)

vadd(vmul(x,y),z) == vmac(x,y,z)

# asymptotic

https://github.com/teorth/estimates
https://en.wikipedia.org/wiki/Big_O_notation 
https://www.andrew.cmu.edu/user/avigad/Papers/bigo.pdf  Formalizing O notation in Isabelle/HOL

eqaulity isn't equality (unless it is?)

f(x) = O(g(x)) is really
O(g)[f]

But to do this is to ignore the wisdom of the notation perhaps. because formalists can't fit a notation into their rigid paradgims doesn't mean its bad

I could do custom overloading of == to mean asymptotic.
Didn't I have some idea about O ~ id



nonstandard reals
Kind of a pain in the ass to do it this way. Is the injectivity it offers us even that useful?



In [ ]:
NonR0 = smt.DeclareSort("NonR0")
NonR = kd.Inductive("NonR")
NonR.declare("Std", ("R", smt.RealSort()))
NonR.declare("NonStd", ("Nr", NonR0))
NonR = NonR.create()

# std predicate is now is_Std
x,y = smt.Consts("x y", NonR)
add = kd.notation.add.define([x,y],
                             kd.cond(
                                (smt.And(smt.is_Std(x), smt.is_Std(y)), x + y),
                                (smt.And(smt.is_Std(x), smt.is_NonStd(y)), x + y),
                                (smt.And(smt.is_NonStd(x), smt.is_Std(y)), x + y),
                                (smt.And(smt.is_NonStd(x), smt.is_NonStd(y)), x + y
                             )





# pysmt
I never _really_ gave pysmt a fair shake.
It doesn't support lambda. That is a pretty tough blow



In [ ]:
from pysmt.shortcuts import Symbol, And, Not, is_sat

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
is_sat(f)
g = f.substitute({varB: varA})
is_sat(g)

False

# lambda unify


In [ ]:
def unify_huet():
    todo = []
    frozen = [] #flexflex?
    subst = {}
    while True:
        while todo:
            current = todo.pop()
            # do unification.

        for p in frozen:
            if flexflex(p):
                continue
            else:
                frozen.remove(p)
                todo.append(p)
                break
        else:
            return subst, frozen
    
        

    

# GAT / refinement
see types.ipynb also


The property of being a proper telescoping mapping requires an smt solve?

why bother teelscoping? Isabelle pure has no telescoping effect? CVody says that helps build model but did not elaborate
https://argo.matf.bg.ac.rs/events/2009/fatpa2009/slides/Wenzel_PureLogicalReasoning%20in%20Isabelle-Isar.pdf

https://arxiv.org/pdf/1911.00399 hott in pure
https://arxiv.org/pdf/cs/9301105 pualson foundation of generic theorem prover


So a conditional judgement can be seen as a fiber just as much as types can (?)
A /\ B /\ C ===> D

|- goal(p)  as a marker. Always provable goal(p) == True
 
unify(pf1, pf2)

Rather than an equalizer, a pullback. t1 as a substituion mapping anyway?
G1 -> G2
G3 -> G2
Two telescope mappings that go to common context G2. Can I build a pullback?




In [ ]:
from dataclasses import dataclass

@dataclass
class Telescope():
    """ x | A(x), y | B(x,y), ... """
    vs : list[smt.ExprRef]
    preds : list[smt.BoolRef]
    # preds : list[kd.Proof] ? |- ForAll([x], P1), ForAll([x,y], P1 => P2), ...  
    # No but the context should be proof recievining, not proof producing.

    def expr(self, P):
        print(self.vs, self.preds, P)
        return kd.QForAll(self.vs, smt.And(self.preds), P)
    


x,y,z = smt.Reals("x y z")
Telescope([x,y], [x > 0, y > x])

@dataclass
class TeleSubst():
    dom : Telescope
    cod : Telescope
    subst: dict[smt.ExprRef, smt.ExprRef]
    # pf : list[smt.Proof] # same length as dom.preds

    def check(self,by=[]):
        pfs = []
        pfs.extend(by)
        for P in self.dom.preds:
            pfs.append(kd.prove(self.cod.expr(smt.substitute(P, *self.subst.items())), by=pfs))
        return pfs
    def __call__(self, P):
        # P is in context dom
        return smt.substitute(P, *self.subst.items())
    def __matmul__(self, other):
        # substutition forms a category
        assert self.dom == other.cod
        TeleSubst(other.dom, self.cod, {smt.substitute(...)})

# allows subtype weakening.
# if it only allowed syntactic substitutions, then we don't really need the proofs

ctx1 = Telescope([x,z], [x >= 0, z >= x])
ctx2 = Telescope([y], [y >= -1])
TeleSubst(ctx1, ctx2, {x: y + 1, z : y + 10}).check()


# is there a unification procedure for making TeleSubst?
# G1 |- t1 =? t2 -| G2
# result is G1 -> G2 substitution? It's assymmetric?
# doesn't seem like the right shape.



[y] [y >= -1] y + 1 >= 0
[y] [y >= -1] y + 10 >= y + 1


[|- ForAll(y, Implies(And(y >= -1), y + 1 >= 0)),
 |- ForAll(y, Implies(And(y >= -1), y + 10 >= y + 1))]

In [ ]:
from kdrag.all import *

def subst(pf : kd.Proof, vs : list[smt.ExprRef], subst : list[smt.ExprRef]) -> kd.Proof:
    vs1, ab = kd.kernel.herb(smt.ForAll(vs, smt.substitute_vars(pf.thm.body(), *reversed(subst))))
    a = kd.kernel.instan([smt.substitute(t, *zip(vs, vs1)) for t in subst], pf)
    return kd.kernel.modus(ab, a)
x,y,z = smt.Reals("x y z")
p = kd.prove(smt.ForAll([x,z], smt.And(z == z, x == x)))
subst(p, [y, z], [y + 1, z])

def weaken(pf, x):


def unify(pf1, pf2) -> Optional[kd.Proof]:
    # unify the two proofs forall x, p1(x) and forall y, p2(y)
    # return a new proof
    vs1, p1 = kd.utils.open_binder(pf1.thm)
    vs2, p2 = kd.utils.open_binder(pf2.thm)
    vs = vs1 + vs2
    subst = kd.utils.unify(vs1+vs2,p1,p2)
    if subst is not None:
        vs = set(vs) - set(subst.keys()) # maybe keep 'em ordered
        return subst_tac(pf1, vs, )

def build_telescope(pf : kd.Proof, vs, hyps, conc):
    # find a substitution based on unifying conc


        

def chain(pf1, pf2):
    # decompose into rules.
    # unify
    # rebuild






|- ForAll([y, z], And(z == z, y + 1 == y + 1))

In [ ]:
import kdrag.reflect as reflect
from kdrag.all import *
x = smt.Bool("x")
@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# verilog extract

Silviu makes a good point.
If you're going to be comparing test case outputs,
No need to 



In [ ]:


x,y,z = smt.Bools("x y z")
a,b,c = smt.BitVecs("a b c", 8)
print(to_verilog("test", [x,y], {z : x & y}))
print(to_verilog("test", [a,b], {z : a & b + 7 != b}))
#(a & b != b).decl().name()

def test():
    with open("/tmp/test.v", "w") as f:
        f.write(to_verilog("test", [x,y], {z : x & y}))
    


// Generated by knuckledragger
module test (
   input x,
   input y,
   output z
);
assign z = (x && y);
endmodule

// Generated by knuckledragger
module test (
   input [7:0] a,
   input [7:0] b,
   output z
);
assign z = ((a & (b + 8'd7)) != b);
endmodule



To say that a verilog file has a property is an axiom schema?

In [ ]:
def sby_axiom(modfile, name, ins, outs):
    


# diff




In [ ]:
from kdrag.all import *

def diff(e, x):
    if e.eq(x):
        return 1
    elif smt.is_value(e):
        return 0
    elif smt.is_add(e):
        return smt.Sum([diff(c,x) for c in e.children()])
    elif e.decl() == real.sin:
        return real.cos(e) * diff(e, x)
    elif e.decl() == real.cos:
        return -real.sin(e) * diff(e, x)
    elif e.decl() == real.exp:
        return real.exp(e) * diff(e, x)
    
    


# knuckelproblems


In [ ]:


from kdrag.all import *
# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

"""
add = kd.define("add", [n,m], 
    n.match_(
        (Nat.Zero, m),
        (Nat.Succ(n), Nat.Succ(add(n, m)))
))
add.defn
"""
add.defn


|- ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!6)))

In [ ]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# vectors

Bitvector operations

V4 style of 


In [ ]:
BV32 = smt.BitVecSort(32)




# Type resgistry

Some 

add_assoc
add_comm
mul_assoc
mul_comm


COuld reaplce sortdispatch using T.add, T.mul and just have ExprRef.__add__ = lambda x,y: x.sort().add(x,y)



BoolSort
RealSort
StringSort
etc


In [ ]:
from kdrag.all import *
T = smt.DeclareSort("T")
T1 = smt.DeclareSort("T")
id(T1) == id(T)
T1.get_id() == T.get_id()

True

In [ ]:
from kdrag.all import *
smt.sort_registry
import kdrag.theories.nat as nat
smt.sort_registry
import kdrag.theories.bitvec as bv
smt.sort_registry

{2147483659: Int,
 2147483648: Bool,
 2147483658: Real,
 2147483815: NatI,
 2147483814: Nat,
 2147483704: BitVec(1),
 2147483711: BitVec(8),
 2147483898: Seq(BitVec(1)),
 2147483841: BitVecN}

In [ ]:
# hash cons the types so that we can tag useful stuff on them
types = {}

def DeclareSort(name):
    if name not in types:
        types[name] = type(name, (object,), {})
    return types[name]


# Schematic Vars
Explicitly pulling schematic variables out, may let me refer to them in subproofs (How?)

   G |- A
------------- weaken
     G, v fresh |- A
Well that's annoying.

app(forall x, A, t)




forall x, A  

Maybe I should consider this as A Proof extension rather than proof replacement, likke refinement types


```
Gam |- t   x not in t
-----------
Gam, x |- t 

```


If I merged forall intro and removal


This is just |- forall x, P(x) but with some extra junk.

capture avoiding

https://ncatlab.org/nlab/show/substitution

https://en.wikipedia.org/wiki/Admissible_rule
https://cstheory.stackexchange.com/questions/54600/admissible-rules-in-dependent-type-theory

Gam |- A then sigma Gam |- sigma A


The point was that

prove(t, )

def cprove(vs, p, by=)


a curried form of backward proof tracking.
modus for leaves
comp for moves.
but yeah, split
A /\ B
(A => B => A /\ B)
(A => A \/ B)
(B => A \/ B)

A => B => C

but then aplpying move in context is annoying. Hmm.
And I don't give a shit about anything except quantifier moves so it's wasteful effort.

Maybe the callback method is easier. But it'll have so many expensive calls


def intros():
   




In [ ]:
def modus(ab, a):
    assert smt.is_implies(ab) and ab.arg(0).eq(a)
    assert isinstance(ab, Proof) and isinstance(a, Proof)
    return Proof(ab.arg(1), reasons=[ab, a])

def impl_refl(a):
    return Proof(smt.Implies(a,a))

def comp(ab, bc): # cut sort of
    assert smt.is_implies(ab) and smt.is_implies(bc) and ab.arg(1).eq(bc.arg(0))
    assert isinstance(ab, Proof) and isinstance(bc, Proof)
    return Proof(smt.Implies(ab.arg(0), bc.arg(1)), reasons=[ab, bc])


class Lemma():
    self.backward_proof = impl_refl(goal)
    self.topgoal = goal
    self.curgoal = goal
    

def qed():
    assert self.backwardproof.thm.eq(self.topgoal)
    return self.backwardproof
def fixes():
    herb(self.topgoal)
    self.backwardproof = comp(  , self.backwardproof)(modus(ab,a)))
def prove():
    p = self.prove(curgoal)
    self.backwardproof = modus(self.backwardproof, self.proof)
def intros():
    self.curgoal = self.curgoal.arg(0)
    # goal is unchanged?
    # (a -> b) -> topgoal. uhhh. G -> 
    # 
    # /\ (forall vs, hyps => conc) -> topgoal
    # (hyp1 => conc1) => (hyp2 => conc2) => hyp3 => topgoal
    # goal1 => goal2 => topgoal



Goal
  def to_expr
   

                                                


In [ ]:
def deinstan(pf):
    # get substution if last thing was an instan?
    # Super weird idea.

def cprove(
    vs : list[smt.ExprRef],
    thm: smt.BoolRef,
    by = lambda *args: [] #Callable[smt.ExprRefglist[kd.Proof],
    **kwargs
) -> kd.kernel.Proof:
    """Contextual proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(vs, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)



def indprove(v, thm):
    """Inductive proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(v, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)

def indprove(v : smt.ExprRef, thm, by=None, using=None):
    l = kd.Lemma(smt.ForAll(v, thm))
    v1 = l.fix()
    l.induct(v1, using=using)
    l.auto(by=by)
    return l.qed()

In [ ]:
def subst(vs, p : kd.Proof, subst):
    assert isinstance(p, kd.Proof)
    vs1, body = kd.utils.open_binder_unhygienic(p.fm)
    assert kd.utils.free_in(vs - vs1, body)

# vs

# ok so we can achieve of substutiton by combining herb and instan.
def subst(vs, p : kd.Proof, subst):
    goal = smt.ForAll(vs, smt.substitute(p.fm, *subst.items()))
    l = kd.Lemma(goal)
    vs = l.fixes()
    l.qed(by=p(subst[v] for v in vs))
    #vs, pf = kd.kernel.herb(p)



In [ ]:
class Proof():
    ctx : dict[smt.SortRef, int] # How many de bruijn indices are allowed?
    fm : smt.BoolRef
    

In [ ]:
class Proof():
    vs : list[smt.ExprRef] # ctx
    fm : smt.BoolRef | smt.QuantifierRef
    reasons : list[object]
    def __call__(self, *args, vs=[]):
        smt.substitute(self.fm, *zip(self.vs, args))

def free(vs, fm):
    #fm2 = smt.substitute(fm, *[(v, smt.FreshConst(v.sort())) for v in vs])
    return smt.Lambda(vs, fm).body().eq(fm)
    #return fm2.eq(fm)

def lift(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    return Proof(
        vs = [],
        fm = p.fm,
        reasons = [p]
    )
def lift_forall(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    return Proof(
        vs = vs,
        fm = fm,
        reasons = [p]
    )
def lift_free(ctx, p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    assert all(v in ctx for v in p.vs)
    assert free(ctx, p.fm)
    return Proof(
        vs = ctx,
        fm = p.fm,
        reasons = [p]
    )

"""
  Gam |- forall x, P
---------------
 Gam, x |- P

 This is reversed intro rule. Feels odd.
"""
def open(p : Proof) -> Proof:
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    Proof(p.vs + vs, fm, [p])

def intro(n, p : Proof):
    return Proof(p.vs[n:], smt.ForAll(p.vs[:n], p.fm), [p])

def extend(vs, p : Proof) -> Proof:
    s = smt.Solver()
    s.add(smt.ForAll(p.vs, p.fm))
    s.add(smt.Not(smt.ForAll(p.vs + vs, p.fm)))
    res = s.check()
    if res == smt.unsat:
        return Proof(
            vs = p.vs + vs,
            fm = p.fm,
            reasons = [p]
        )
    else:
        raise Exception("Unknown result from solver")


def subst(p : Proof, ctx : list[smt.ExprRef], subst : dict[smt.ExprRef, smt.ExprRef]):
    assert isinstance(p, Proof)
    assert all(v in p.vs for v in  subst.keys())
    assert free(ctx - p.ctx, p.fm)
    #fm = smt.substitute(p.fm, *[(v, smt.FreshConst(v.sort())) for v in ctx])
    fm = smt.substitute(p.fm, *subst.items())
    # I need to check that ctx2 does not appear in fm. Otherwise I could |- even(zero) ---> zero |- even(zero)
    return Proof(
        vs = ctx,
        fm = fm,
        reasons = [p]
    )


"""
G |- p1    G |- p2 ... 
-------
    G |- p

"""

def prove(ctx, expr, by=None):
    allctx = set(ctx)
    for p in by:
        assert isinstance(p, Proof)
        assert p.ctx == ctx
    s.add(smt.Not(smt.ForAll([ctx], smt.Implies(smt.And(hyp.fm for hyp in by), expr))))
    s.check()




Ok so yes, any forall infecting the lemmas is a problem.
And some things get foralls because its hard an annoying 

qed should 


In [ ]:
from kdrag.all import *
import kdrag.theories.real as real
x,y = smt.Reals("x y")
cos = real.cos
sin = real.sin
#os_diff = kd.prove(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)), [real.cos_add, real.cos_neg, real.sin_neg])

_l = kd.Lemma(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)))
_x, _y = _l.fixes()
_l.symm()
_l.eq(cos(_x + (-_y)))
#_l.rw(kd.prove(cos(_x - _y) == cos(_x + (-_y))))
#_l.rw(kd.prove(smt.ForAll([x,y], x - y == (x + (-y))))(_x, _y))
_l.rw(real.cos_add(_x, -_y))
#_l.auto(by=[real.cos_add, real.sin_add])
_l.auto(by=[real.cos_neg(_y), real.sin_neg(_y)])
_l.lemmas
_l.qed()
cos_diff = _l.lemmas


# Homomorphism theorems


In [ ]:
from kdrag.all import *
import kdrag.property as prop
import kdrag.theories.algebra.group as group

A = smt.DeclareSort('A')
def Group(A):
    mul = smt.Function('mul', A, A, A)
    inv = smt.Function('inv', A, A)
    id_ = smt.Const('id', A)
    x,y,z = smt.Consts('x y z', A)
    kd.notation.mul.register(A, mul)
    kd.notation.invert.register(A, inv)
    assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
    id_left = kd.axiom(smt.ForAll([x], id_ * x == x))
    inv_left = kd.axiom(smt.ForAll([x], inv(x) * x == id_))
    return id_, assoc, id_left, inv_left


B = smt.DeclareSort('B')
group.Group.register(A)
idA, assocA, id_leftA, inv_leftA = Group(A)
idB, assocB, id_leftB, inv_leftB = Group(B)

h = smt.Function('h', A, B)
def homo(id_, h):
    A = h.domain(0)
    B = h.range()
    x,y,z = smt.Consts("x y z", A)
    mul = smt.ForAll([x,y], h(x * y) == h(x) * h(y))
    id_ = h(idA) == idB
    inv = smt.ForAll([x], h(~x) == ~h(x))
    return mul, id_, inv
homo(h)





ValueError: ('Not registered in typeclass', (A,))

Universe of groups.

I was saying construct it as permuations  Int -> Int

Nontrivial
But we could also driectly axiomatize

* is composition
id is  x -> x
homomorphisms need to... be careful here. Only push homo through on particular Group?
SetGrp 





In [ ]:

Grp = smt.DeclareSort("Grp")
SetGrp = set_.SetSort(Grp)
inv = smt.Function("inv", Grp, Grp)
# inv = smt.Function("inv", GrpSet, GrpSet)
mul = smt.Function("mul", Grp, Grp) # We just leave mul un

Closed = smt.QForAll([x,y, S], S[x], S[y], S[x * y], S[inv(x)])

smt.QForAll([S], Closed[S], S[x], S[y], S[z],  x*(y*z) == (x * y) * z) # Is there a model where mul can be associative and always defined? 
smt.QForAll([S], Closed[S], S[x], inv(x) * x == x)  # maybe unconditionally actually
smt.QForAll([S], id_ * x == x) # also unconditiojnality